In [1]:
%%bash
pip3 install user_agents
pip3 install tqdm

  Running setup.py bdist_wheel for user-agents: started
  Running setup.py bdist_wheel for user-agents: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4c/36/d8/6b28dfb933d4397e28dcf7b6d426b85808a22eed8c4107e6c2
Successfully built user-agents


In [2]:
import numpy as np
import pandas as pd
import pickle
import urllib
from tqdm import tqdm, trange, tqdm_pandas
import os
from user_agents import parse
import datetime
import time
pd.set_option('display.max_columns', 1000)

| name          |meaning        | 
| ------------- |:-------------| 
|**data    **      |data_final from data_manipulation table|
|**data_base**    |data with 'ip','city','country' 3 columns dropped|

---
# Import Data:

In [5]:
%%time
# initial inner join data
# df=pickle.load(open( "../Data/df.p", "rb" ) )
# dataframe dropping a few columns
# df0=pickle.load(open( "../Data/df0.p", "rb" ) )
# reorganized main dataframe
with open( "../Data/data_final.p", "rb" )  as f:
    data=pickle.load(f)

CPU times: user 4.34 s, sys: 1.8 s, total: 6.14 s
Wall time: 6.14 s


---
# Feature Engineering Part 1 (Preprocess):
- **df is the final base dataframe we would use**
- **Each subsection of Feature Engineering would update a column of df_groupby2 (can rerun without reload)**

## Core dataframe: df_groupby2

**Have a look at df**

In [6]:
num_row=3
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
data.iloc[list,:]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,None,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,None,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB
1469182,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.554,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469183,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:09,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Ftrave...,NaT,2017-06-12 14:28:09.550,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB
1469184,52433,4,1tz5t8tsg2bdj5c3w6i1r5ufu,3a131efaa20b6a197508313f0eb266c4,2017-06-12 14:28:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Ftravel,NaT,NaT,None,3-5nt New York Break with Flights,20%,369,08:08:31:55,sue_terryshannon@yahoo.co.uk,None,1497276983453,1497276973316,369,False,None,False,2017-06-12 15:15:54,2017-06-12 15:28:26,1497279605,86.6.191.124,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_2_1+like...,Oxford,GB


**Drop IP, City and Country in data**

In [7]:
data_base=data.drop(['ip_address','city','country'],axis=1)
data_base.drop_duplicates().shape

(1427002, 27)

In [13]:
print('Pencentage of duplication cause by geolocation:\n',(1469184-1427002)/1469184*100,'%')

Pencentage of duplication cause by geolocation:
 2.8711175727478655 %


---
## 1. Dwell time

- **df_groupby2 is the table having each line to be each session of each customer**

In [94]:
%%time
# .nth(0) would choose the nth term in each group.
#  GroupBy.nth(n, dropna=None), if dropna = 'any', it would drop any non-null cell in dataframe before groupby
df_groupby2 = df.groupby(['id','num_sess'],sort=False).nth(0)
last_browsing_time=pd.to_datetime(df_groupby2.cart_lastTime.values)
first_browsing_time=pd.to_datetime(df_groupby2.datetime.values)
df_groupby2['dwell_time']=last_browsing_time-first_browsing_time

CPU times: user 1.47 s, sys: 198 ms, total: 1.67 s
Wall time: 3.17 s


In [95]:
df.columns

Index(['id', 'num_sess', 'session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country'],
      dtype='object')

In [99]:
df_groupby2=df_groupby2[['session_id', 'finger_print', 'datetime', 'days',
       'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country','dwell_time']]
df_groupby2.loc[[2,3],:]

session_id                      finger_print  \
id num_sess                                                                 
2  1          qvsw8ty43cbb15tu52ujywjs3  e5f3ca057208f51a2cacae58ff95d757   
   2           xe1kopsb1o573zhid3i51eds  e5f3ca057208f51a2cacae58ff95d757   
   3         184grb3zg9udt1scfxr2697n5b  e5f3ca057208f51a2cacae58ff95d757   
   4          11expdzyaf01l4rt3b953me14  e5f3ca057208f51a2cacae58ff95d757   
3  1         1fxh0wekr0wq81on76gwu04gap  c91effe8aa9659c7f41e1fcab45c7db6   
   2           tf3ruinsik6kmfdxx2bzy6mz  c91effe8aa9659c7f41e1fcab45c7db6   
   3         1wjziazqo6cjz1tzkyw11s34kw  c91effe8aa9659c7f41e1fcab45c7db6   

                       datetime  days referrer  \
id num_sess                                      
2  1        2017-05-26 13:25:27    26            
   2        2017-06-09 13:20:22     9            
   3        2017-06-12 08:32:26    12            
   4        2017-06-14 13:50:01    14            
3  1        2017-06-03 18:04:17     3            
   2        2017-06-05 23:04:29     5            
   3        2017-06-13 17:19:18    13            

                                                      page_url  \
id num_sess                                                      
2  1         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
   2         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   4         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
3  1         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   2         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

              product_viewed_browse cart_started basket_paid_browsing  \
id num_sess                                                             
2  1        2017-05-26 13:25:20.268          NaT                 None   
   2        2017-06-09 13:20:30.130          NaT                 None   
   3        2017-06-12 08:32:43.033          NaT                 None   
   4        2017-06-14 13:49:41.154          NaT                 None   
3  1        2017-06-03 17:03:45.208          NaT                 None   
   2        2017-06-05 22:04:00.532          NaT                 None   
   3        2017-06-13 16:18:54.391          NaT                 None   

                                                  product_name product_save  \
id num_sess                                                                   
2  1                        3 5 or 7nt Algarve Break & Flights          26%   
   2         7nt 4* All-Inclusive Croatia & Flights - Summe...          27%   
   3                 Modern Shaggy Rug - 10 Colours & 6 Sizes!          64%   
   4              7nt All-Inclusive Mallorca Holiday & Flights          35%   
3  1                              Kingfisher Wooden Bird Hotel          81%   
   2                                Personalised Photo Blanket          87%   
   3                             Diamante Sandals - 3 Colours!          70%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
2  1                   69       06:09:34:40  None      None  1495805120891   
   2                  229       04:09:39:29  None      None  1497014430749   
   3                    9       03:14:27:15  None      None  1497256364377   
   4                  259       06:09:09:13  None      None  1497448181772   
3  1                 6.99       02:05:55:20  None      None  1496509437864   
   2                   12       04:00:50:03  None      None  1496700477983   
   3                    8       02:06:41:05  None      None  1497370739681   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
2  1                       None    69            False         None   
   

- **save df_groupby2**

In [100]:
%%time
df_groupby2 = df_groupby2.rename(columns = {'datetime':'date_time'})
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

CPU times: user 2.08 s, sys: 471 ms, total: 2.55 s
Wall time: 3.61 s


In [101]:
num_row=3
list=np.concatenate((range(0,num_row),range(-num_row,0)),axis=0)
df_groupby2.iloc[list,:]

session_id                      finger_print  \
id    num_sess                                                                 
1     1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
      2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
      3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   
52433 2           vxhv5kka1iu93rtmz4g6osc4  de6f52700a2fefcd15e803abbe23cde1   
      3          15ox967rdf7q3yjn65eedevsh  de6f52700a2fefcd15e803abbe23cde1   
      4          1tz5t8tsg2bdj5c3w6i1r5ufu  3a131efaa20b6a197508313f0eb266c4   

                         date_time  days  \
id    num_sess                             
1     1        2017-05-19 08:02:53    19   
      2        2017-05-19 13:51:59    19   
      3        2017-06-02 21:40:06     2   
52433 2        2017-06-12 14:03:52    12   
      3        2017-06-12 14:03:52    12   
      4        2017-06-12 14:15:41    12   

                                                         referrer  \
id    num_sess                                                      
1     1                                                             
      2                                                             
      3                                                             
52433 2         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      4                                                             

                                                         page_url  \
id    num_sess                                                      
1     1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
52433 2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      3         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      4          https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Flondon   

               product_viewed_browse            cart_started  \
id    num_sess                                                 
1     1                          NaT 2017-05-19 08:02:51.292   
      2                          NaT 2017-05-19 08:02:51.292   
      3                          NaT 2017-06-02 21:40:06.147   
52433 2                          NaT 2017-06-12 14:03:52.170   
      3                          NaT 2017-06-12 14:03:52.155   
      4                          NaT                     NaT   

               basket_paid_browsing  \
id    num_sess                        
1     1                        None   
      2                        None   
      3                        None   
52433 2                        None   
      3                        None   
      4                        None   

                                                    product_name product_save  \
id    num_sess                                                                  
1     1                                                     None         None   
      2                                                     None         None   
      3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   
52433 2           9-Seater XL Grey Rattan Garden Dining Sofa Set          48%   
      3           9-Seater XL Grey Rattan Garden Dining Sofa Set          48%   
      4                        3-5nt New York Break with Flights          20%   

               product_price product_time_left                         email  \
id    num_sess                                                                 
1     1                 None              None                          None   
      2                 None              None                          None   
      3                    9       03:01:20:04                          None   
52433 2               509.00       01:08:56:14  sue_terry

---
## 2. Weekday

**Load df_groupby2: **

In [367]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2 = pickle.load(f)

CPU times: user 644 ms, sys: 209 ms, total: 853 ms
Wall time: 846 ms


In [102]:
# Monday is 0, Sunday is 6
df_groupby2['weekday'] = df_groupby2.date_time.apply(lambda x: x.dayofweek).values

In [114]:
df_groupby2.date_time[0]

Timestamp('2017-05-19 08:02:53')

In [103]:
df_groupby2[:3]

session_id                      finger_print  \
id num_sess                                                                 
1  1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
   2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
   3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   

                      date_time  days referrer  \
id num_sess                                      
1  1        2017-05-19 08:02:53    19            
   2        2017-05-19 13:51:59    19            
   3        2017-06-02 21:40:06     2            

                                                      page_url  \
id num_sess                                                      
1  1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

            product_viewed_browse            cart_started  \
id num_sess                                                 
1  1                          NaT 2017-05-19 08:02:51.292   
   2                          NaT 2017-05-19 08:02:51.292   
   3                          NaT 2017-06-02 21:40:06.147   

            basket_paid_browsing  \
id num_sess                        
1  1                        None   
   2                        None   
   3                        None   

                                                 product_name product_save  \
id num_sess                                                                  
1  1                                                     None         None   
   2                                                     None         None   
   3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
1  1                 None              None  None      None  1495180971292   
   2                 None              None  None      None  1495180971292   
   3                    9       03:01:20:04  None      None  1496439606147   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
1  1                       None  None            False         None   
   2                       None  None            False         None   
   3                       None     9            False         None   

            email_click_through       cart_firstTime        cart_lastTime  \
id num_sess                                                                 
1  1                       True  2017-05-19 09:02:53  2017-05-19 09:02:53   
   2                       True  2017-05-19 14:51:59  2017-05-19 14:51:59   
   3                      False  2017-06-02 22:40:06  2017-06-02 22:40:06   

            basketInfoTime      ip_address  \
id num_sess                                  
1  1            1495182843  77.103.157.226   
   2            1495203782  77.103.157.226   
   3            1496441524  77.103.157.226   

                                                    user_agent  city country  \
id num_sess                                                                    
1  1         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   2         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   3         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   

             dwell_time  weekday  
id num_sess                       
1  1           01:00:00        4  
   2           01:00:00        4  
   3           01:00:00        4

**Save df_groupby2:**

In [104]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f )

CPU times: user 1.85 s, sys: 466 ms, total: 2.32 s
Wall time: 3.36 s


---
## 3. Month

**Load df_groupby2:**

In [371]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2 = pickle.load(f)

CPU times: user 681 ms, sys: 213 ms, total: 894 ms
Wall time: 891 ms


**Move days to end of column**

In [105]:
cols=df_groupby2.columns.tolist()
cols

['session_id',
 'finger_print',
 'date_time',
 'days',
 'referrer',
 'page_url',
 'product_viewed_browse',
 'cart_started',
 'basket_paid_browsing',
 'product_name',
 'product_save',
 'product_price',
 'product_time_left',
 'email',
 'Post_Code',
 'cart_startTime',
 'product_viewed_cart',
 'total',
 'basket_paid_cart',
 'payment_type',
 'email_click_through',
 'cart_firstTime',
 'cart_lastTime',
 'basketInfoTime',
 'ip_address',
 'user_agent',
 'city',
 'country',
 'dwell_time',
 'weekday']

In [106]:
# move days to end of the column
cols=['session_id',
 'finger_print',
 'date_time',
 'referrer',
 'page_url',
 'product_viewed_browse',
 'cart_started',
 'basket_paid_browsing',
 'product_name',
 'product_save',
 'product_price',
 'product_time_left',
 'email',
 'Post_Code',
 'cart_startTime',
 'product_viewed_cart',
 'total',
 'basket_paid_cart',
 'payment_type',
 'email_click_through',
 'cart_firstTime',
 'cart_lastTime',
 'basketInfoTime',
 'ip_address',
 'user_agent',
 'city',
 'country',
 'dwell_time',
 'weekday','days']

In [107]:
df_groupby2=df_groupby2[cols]

In [115]:
df_groupby2[:3]

session_id                      finger_print  \
id num_sess                                                                 
1  1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
   2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
   3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   

                      date_time referrer  \
id num_sess                                
1  1        2017-05-19 08:02:53            
   2        2017-05-19 13:51:59            
   3        2017-06-02 21:40:06            

                                                      page_url  \
id num_sess                                                      
1  1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

            product_viewed_browse            cart_started  \
id num_sess                                                 
1  1                          NaT 2017-05-19 08:02:51.292   
   2                          NaT 2017-05-19 08:02:51.292   
   3                          NaT 2017-06-02 21:40:06.147   

            basket_paid_browsing  \
id num_sess                        
1  1                        None   
   2                        None   
   3                        None   

                                                 product_name product_save  \
id num_sess                                                                  
1  1                                                     None         None   
   2                                                     None         None   
   3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
1  1                 None              None  None      None  1495180971292   
   2                 None              None  None      None  1495180971292   
   3                    9       03:01:20:04  None      None  1496439606147   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
1  1                       None  None            False         None   
   2                       None  None            False         None   
   3                       None     9            False         None   

            email_click_through       cart_firstTime        cart_lastTime  \
id num_sess                                                                 
1  1                       True  2017-05-19 09:02:53  2017-05-19 09:02:53   
   2                       True  2017-05-19 14:51:59  2017-05-19 14:51:59   
   3                      False  2017-06-02 22:40:06  2017-06-02 22:40:06   

            basketInfoTime      ip_address  \
id num_sess                                  
1  1            1495182843  77.103.157.226   
   2            1495203782  77.103.157.226   
   3            1496441524  77.103.157.226   

                                                    user_agent  city country  \
id num_sess                                                                    
1  1         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   2         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   3         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   

             dwell_time  weekday  days  
id num_sess                             
1  1           01:00:00        4    19  
   2           01:00:00        4    19  
   3           01:00:00        4     2

In [116]:
df_groupby2['month'] = df_groupby2.date_time.apply(lambda x: x.month).values

In [118]:
df_groupby2[:3]

session_id                      finger_print  \
id num_sess                                                                 
1  1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
   2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
   3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   

                      date_time referrer  \
id num_sess                                
1  1        2017-05-19 08:02:53            
   2        2017-05-19 13:51:59            
   3        2017-06-02 21:40:06            

                                                      page_url  \
id num_sess                                                      
1  1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

            product_viewed_browse            cart_started  \
id num_sess                                                 
1  1                          NaT 2017-05-19 08:02:51.292   
   2                          NaT 2017-05-19 08:02:51.292   
   3                          NaT 2017-06-02 21:40:06.147   

            basket_paid_browsing  \
id num_sess                        
1  1                        None   
   2                        None   
   3                        None   

                                                 product_name product_save  \
id num_sess                                                                  
1  1                                                     None         None   
   2                                                     None         None   
   3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
1  1                 None              None  None      None  1495180971292   
   2                 None              None  None      None  1495180971292   
   3                    9       03:01:20:04  None      None  1496439606147   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
1  1                       None  None            False         None   
   2                       None  None            False         None   
   3                       None     9            False         None   

            email_click_through       cart_firstTime        cart_lastTime  \
id num_sess                                                                 
1  1                       True  2017-05-19 09:02:53  2017-05-19 09:02:53   
   2                       True  2017-05-19 14:51:59  2017-05-19 14:51:59   
   3                      False  2017-06-02 22:40:06  2017-06-02 22:40:06   

            basketInfoTime      ip_address  \
id num_sess                                  
1  1            1495182843  77.103.157.226   
   2            1495203782  77.103.157.226   
   3            1496441524  77.103.157.226   

                                                    user_agent  city country  \
id num_sess                                                                    
1  1         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   2         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   3         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   

             dwell_time  weekday  days  month  
id num_sess                                    
1  1           01:00:00        4    19      5  
   2           01:00:00        4    19      5  
   3           01:00:00        4     2      6

**Save df_groupby2:**

In [119]:
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

---
## 4. Time of Day
**Session start time.**

**Load df_groupby2:**

In [379]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2 = pickle.load(f)

CPU times: user 674 ms, sys: 198 ms, total: 872 ms
Wall time: 868 ms


In [58]:
# df1[:1].date_time.values
df_groupby2.date_time.values

array(['2017-05-19T08:02:53.000000000', '2017-05-19T13:51:59.000000000',
       '2017-06-02T21:40:06.000000000', ...,
       '2017-06-12T14:03:52.000000000', '2017-06-12T14:03:52.000000000',
       '2017-06-12T14:15:41.000000000'], dtype='datetime64[ns]')

**transform the data to timestamp form**

In [120]:
a=pd.to_datetime(df_groupby2[:1].date_time.values)
a[0]

Timestamp('2017-05-19 08:02:53')

In [121]:
a[0].time()

datetime.time(8, 2, 53)

In [122]:
type(df_groupby2.dwell_time[0])

pandas.tslib.Timedelta

In [130]:
tmp=df_groupby2.date_time.apply(lambda x: pd.to_datetime(x).time()).values
df_groupby2['time_of_day']=tmp # this is an array

In [132]:
df_groupby2[:3]

session_id                      finger_print  \
id num_sess                                                                 
1  1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
   2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
   3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   

                      date_time referrer  \
id num_sess                                
1  1        2017-05-19 08:02:53            
   2        2017-05-19 13:51:59            
   3        2017-06-02 21:40:06            

                                                      page_url  \
id num_sess                                                      
1  1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

            product_viewed_browse            cart_started  \
id num_sess                                                 
1  1                          NaT 2017-05-19 08:02:51.292   
   2                          NaT 2017-05-19 08:02:51.292   
   3                          NaT 2017-06-02 21:40:06.147   

            basket_paid_browsing  \
id num_sess                        
1  1                        None   
   2                        None   
   3                        None   

                                                 product_name product_save  \
id num_sess                                                                  
1  1                                                     None         None   
   2                                                     None         None   
   3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
1  1                 None              None  None      None  1495180971292   
   2                 None              None  None      None  1495180971292   
   3                    9       03:01:20:04  None      None  1496439606147   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
1  1                       None  None            False         None   
   2                       None  None            False         None   
   3                       None     9            False         None   

            email_click_through       cart_firstTime        cart_lastTime  \
id num_sess                                                                 
1  1                       True  2017-05-19 09:02:53  2017-05-19 09:02:53   
   2                       True  2017-05-19 14:51:59  2017-05-19 14:51:59   
   3                      False  2017-06-02 22:40:06  2017-06-02 22:40:06   

            basketInfoTime      ip_address  \
id num_sess                                  
1  1            1495182843  77.103.157.226   
   2            1495203782  77.103.157.226   
   3            1496441524  77.103.157.226   

                                                    user_agent  city country  \
id num_sess                                                                    
1  1         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   2         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   3         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   

             dwell_time  weekday  days  month time_of_day  
id num_sess                                                
1  1           01:00:00        4    19      5    08:02:53  
   2           01:00:00        4    19      5    13:51:59  
   3           01:00:00        4     2      6    21:40:06

**Save df_groupby2:**

In [133]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

CPU times: user 2.53 s, sys: 447 ms, total: 2.98 s
Wall time: 4.08 s


---
## 5. Cart time
In wowcher data, there's no proper cart. Possible useful features could be cart dwell time (cart time), more detailed dwell time of a certain product page etc.

**Load df_groupby2:**

In [137]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2 = pickle.load(f)

CPU times: user 1.02 s, sys: 303 ms, total: 1.32 s
Wall time: 1.52 s


In [139]:
# cart_first_time and cart_last_time are all same during a session
cart_first_time=pd.to_datetime(df_groupby2.cart_firstTime.values)
cart_last_time=pd.to_datetime(df_groupby2.cart_lastTime.values)
# assign new columns:
df_groupby2['cart_time']=cart_last_time-cart_first_time

In [142]:
# didn't use 'inplace= True', so it is just for display purpose
df_groupby2.loc[[2,3],:]

session_id                      finger_print  \
id num_sess                                                                 
2  1          qvsw8ty43cbb15tu52ujywjs3  e5f3ca057208f51a2cacae58ff95d757   
   2           xe1kopsb1o573zhid3i51eds  e5f3ca057208f51a2cacae58ff95d757   
   3         184grb3zg9udt1scfxr2697n5b  e5f3ca057208f51a2cacae58ff95d757   
   4          11expdzyaf01l4rt3b953me14  e5f3ca057208f51a2cacae58ff95d757   
3  1         1fxh0wekr0wq81on76gwu04gap  c91effe8aa9659c7f41e1fcab45c7db6   
   2           tf3ruinsik6kmfdxx2bzy6mz  c91effe8aa9659c7f41e1fcab45c7db6   
   3         1wjziazqo6cjz1tzkyw11s34kw  c91effe8aa9659c7f41e1fcab45c7db6   

                      date_time referrer  \
id num_sess                                
2  1        2017-05-26 13:25:27            
   2        2017-06-09 13:20:22            
   3        2017-06-12 08:32:26            
   4        2017-06-14 13:50:01            
3  1        2017-06-03 18:04:17            
   2        2017-06-05 23:04:29            
   3        2017-06-13 17:19:18            

                                                      page_url  \
id num_sess                                                      
2  1         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
   2         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   4         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
3  1         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   2         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

              product_viewed_browse cart_started basket_paid_browsing  \
id num_sess                                                             
2  1        2017-05-26 13:25:20.268          NaT                 None   
   2        2017-06-09 13:20:30.130          NaT                 None   
   3        2017-06-12 08:32:43.033          NaT                 None   
   4        2017-06-14 13:49:41.154          NaT                 None   
3  1        2017-06-03 17:03:45.208          NaT                 None   
   2        2017-06-05 22:04:00.532          NaT                 None   
   3        2017-06-13 16:18:54.391          NaT                 None   

                                                  product_name product_save  \
id num_sess                                                                   
2  1                        3 5 or 7nt Algarve Break & Flights          26%   
   2         7nt 4* All-Inclusive Croatia & Flights - Summe...          27%   
   3                 Modern Shaggy Rug - 10 Colours & 6 Sizes!          64%   
   4              7nt All-Inclusive Mallorca Holiday & Flights          35%   
3  1                              Kingfisher Wooden Bird Hotel          81%   
   2                                Personalised Photo Blanket          87%   
   3                             Diamante Sandals - 3 Colours!          70%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
2  1                   69       06:09:34:40  None      None  1495805120891   
   2                  229       04:09:39:29  None      None  1497014430749   
   3                    9       03:14:27:15  None      None  1497256364377   
   4                  259       06:09:09:13  None      None  1497448181772   
3  1                 6.99       02:05:55:20  None      None  1496509437864   
   2                   12       04:00:50:03  None      None  1496700477983   
   3                    8       02:06:41:05  None      None  1497370739681   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
2  1                       None    69            False         None   
   2                       None   229            False   

**Save df_groupby2:**

In [143]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

---
## 6. #Page browsed
This is a measurement of how active the customer is.

**Load df_groupby2:**

In [391]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2 = pickle.load(f)

CPU times: user 925 ms, sys: 254 ms, total: 1.18 s
Wall time: 1.18 s


In [144]:
tmp=df[['id','num_sess','page_url']].copy()
df_groupby2['num_browsing']=tmp.groupby(['id','num_sess']).page_url.count()

In [146]:
df_groupby2.loc[[2,3],:]

session_id                      finger_print  \
id num_sess                                                                 
2  1          qvsw8ty43cbb15tu52ujywjs3  e5f3ca057208f51a2cacae58ff95d757   
   2           xe1kopsb1o573zhid3i51eds  e5f3ca057208f51a2cacae58ff95d757   
   3         184grb3zg9udt1scfxr2697n5b  e5f3ca057208f51a2cacae58ff95d757   
   4          11expdzyaf01l4rt3b953me14  e5f3ca057208f51a2cacae58ff95d757   
3  1         1fxh0wekr0wq81on76gwu04gap  c91effe8aa9659c7f41e1fcab45c7db6   
   2           tf3ruinsik6kmfdxx2bzy6mz  c91effe8aa9659c7f41e1fcab45c7db6   
   3         1wjziazqo6cjz1tzkyw11s34kw  c91effe8aa9659c7f41e1fcab45c7db6   

                      date_time referrer  \
id num_sess                                
2  1        2017-05-26 13:25:27            
   2        2017-06-09 13:20:22            
   3        2017-06-12 08:32:26            
   4        2017-06-14 13:50:01            
3  1        2017-06-03 18:04:17            
   2        2017-06-05 23:04:29            
   3        2017-06-13 17:19:18            

                                                      page_url  \
id num_sess                                                      
2  1         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
   2         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   4         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
3  1         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   2         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

              product_viewed_browse cart_started basket_paid_browsing  \
id num_sess                                                             
2  1        2017-05-26 13:25:20.268          NaT                 None   
   2        2017-06-09 13:20:30.130          NaT                 None   
   3        2017-06-12 08:32:43.033          NaT                 None   
   4        2017-06-14 13:49:41.154          NaT                 None   
3  1        2017-06-03 17:03:45.208          NaT                 None   
   2        2017-06-05 22:04:00.532          NaT                 None   
   3        2017-06-13 16:18:54.391          NaT                 None   

                                                  product_name product_save  \
id num_sess                                                                   
2  1                        3 5 or 7nt Algarve Break & Flights          26%   
   2         7nt 4* All-Inclusive Croatia & Flights - Summe...          27%   
   3                 Modern Shaggy Rug - 10 Colours & 6 Sizes!          64%   
   4              7nt All-Inclusive Mallorca Holiday & Flights          35%   
3  1                              Kingfisher Wooden Bird Hotel          81%   
   2                                Personalised Photo Blanket          87%   
   3                             Diamante Sandals - 3 Colours!          70%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
2  1                   69       06:09:34:40  None      None  1495805120891   
   2                  229       04:09:39:29  None      None  1497014430749   
   3                    9       03:14:27:15  None      None  1497256364377   
   4                  259       06:09:09:13  None      None  1497448181772   
3  1                 6.99       02:05:55:20  None      None  1496509437864   
   2                   12       04:00:50:03  None      None  1496700477983   
   3                    8       02:06:41:05  None      None  1497370739681   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
2  1                       None    69            False         None   
   2                       None   229            False   

**Save df_groupby2:**

In [147]:
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

---
## 7. Email providor

**Load df_groupby2:**

In [4]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2 = pickle.load(f)

CPU times: user 862 ms, sys: 372 ms, total: 1.23 s
Wall time: 1.87 s


**Count email:**

In [9]:
print('# email null in df_groupby2 is: \n',sum(df_groupby2.email.apply(lambda x: x== None)))

# email null in df_groupby2 is: 
 57394


In [27]:
tmp=df.sort_values(['id','num_sess','email'], ascending=[True,True,False])
df_email = tmp.groupby(['id','num_sess'],sort=False).nth(0)
print('# email null in df_email is: \n',sum(df_email.email.apply(lambda x: x==None)))
print('size of df_email:\n', df_email.shape[0])

# email null in df_email is: 
 57372
size of df_email:
 220047


**email provider function**

In [22]:
def email_provider(email):  
    email = email.lower()
    return email.split('@')[1].split('.')[0]

In [23]:
email_provider(df_email.email.iloc[91803])

'aol'

In [33]:
tqdm_pandas(tqdm())
df_email.email=df_email.email.progress_apply(lambda x: email_provider(x) if not pd.isnull(x) else 'nun')

0it [00:00, ?it/s]
220047it [00:00, 425773.65it/s]


In [34]:
df_email.loc[[1,2000],:]

Post_Code basketInfoTime basket_paid_browsing basket_paid_cart  \
id   num_sess                                                                  
1    1             None     1495182843                 None            False   
     2             None     1495203782                 None            False   
     3             None     1496441524                 None            False   
     4             None     1497020106                 None            False   
2000 1             None     1495440724                 None            False   
     2             None     1495444504                 None            False   
     3             None     1496659387                 None            False   
     4             None     1496737923                 None            False   
     5             None     1496742245                 None            False   
     6             None     1496846830                 None            False   
     7             None     1496935625                 None            False   
     8             None     1497022327                 None            False   
     9          wn8 6rs     1497452763                 None            False   

                    cart_firstTime        cart_lastTime cart_startTime  \
id   num_sess                                                            
1    1         2017-05-19 09:02:53  2017-05-19 09:02:53  1495180971292   
     2         2017-05-19 14:51:59  2017-05-19 14:51:59  1495180971292   
     3         2017-06-02 22:40:06  2017-06-02 22:40:06  1496439606147   
     4         2017-06-09 15:23:10  2017-06-09 15:23:10  1497018190017   
2000 1         2017-05-22 08:40:09  2017-05-22 08:40:09  1495438809053   
     2         2017-05-22 09:44:00  2017-05-22 09:44:02  1495442640866   
     3         2017-06-05 11:11:31  2017-06-05 11:11:35  1496657491241   
     4         2017-06-06 09:00:36  2017-06-06 09:00:37  1496736035500   
     5         2017-06-06 10:12:50  2017-06-06 10:12:54  1496740370125   
     6         2017-06-07 15:15:21  2017-06-07 15:15:56  1496844921330   
     7         2017-06-08 15:55:28  2017-06-08 15:56:01  1496933721178   
     8         2017-06-09 16:00:16  2017-06-09 16:00:17  1497020416337   
     9         2017-06-14 15:28:03  2017-06-14 15:34:53  1497450483999   

                         cart_started  city country            datetime  days  \
id   num_sess                                                                   
1    1        2017-05-19 08:02:51.292  None      GB 2017-05-19 08:02:53    19   
     2        2017-05-19 08:02:51.292  None      GB 2017-05-19 13:51:59    19   
     3        2017-06-02 21:40:06.147  None      GB 2017-06-02 21:40:06     2   
     4        2017-06-09 14:23:10.017  None      GB 2017-06-09 14:23:10     9   
2000 1        2017-05-22 07:40:09.053  None      GB 2017-05-22 07:40:09    22   
     2        2017-05-22 08:44:02.769  None      GB 2017-05-22 08:44:02    22   
     3        2017-06-05 10:11:35.091  None      GB 2017-06-05 10:11:35     5   
     4        2017-06-06 08:00:37.522  None      GB 2017-06-06 08:00:37     6   
     5        2017-06-06 09:12:51.057  None      GB 2017-06-06 09:12:54     6   
     6        2017-06-07 14:15:56.417  None      GB 2017-06-07 14:15:56     7   
     7        2017-06-08 14:56:00.711  None      GB 2017-06-08 14:56:01     8   
     8        2017-06-09 15:00:17.250  None      GB 2017-06-09 15:00:17     9   
     9        2017-06-14 14:34:53.851  None      GB 2017-06-14 14:34:53    14   

                          email email_click_through  \
id   num_sess                                         
1    1                      nun                True   
     2                      nun                True   
     3                      nun               False   
     4                      nun               False   
2000 1         flowtechnologyuk               False   
     2         flowtechnologyuk                True   
     3         flowtechnologyuk               

In [35]:
df_groupby2.email=df_email.email

In [38]:
df_groupby2.email.loc[52425,:]

id     num_sess
52425  1           gmail
       2           gmail
       3           gmail
       4           gmail
       5           gmail
Name: email, dtype: object

**Save df_groupby2:**

In [39]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

CPU times: user 2.02 s, sys: 366 ms, total: 2.39 s
Wall time: 3.47 s


---
## 8. User_agent

**Load df_groupby2:**

In [64]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2=pickle.load(f)

CPU times: user 1.13 s, sys: 228 ms, total: 1.36 s
Wall time: 1.36 s


**Sample experiment of parsing**

In [70]:
type(df_groupby2['user_agent'])

pandas.core.series.Series

In [71]:
user_agent=df_groupby2['user_agent'].copy()

In [72]:
print(parse(user_agent[0]))
print('\n')
print(parse(user_agent[0]).browser)
print(parse(user_agent[0]).os)
print(parse(user_agent[0]).device)

iPhone / iOS 10.3.1 / Mobile Safari 10.0


Browser(family='Mobile Safari', version=(10, 0), version_string='10.0')
OperatingSystem(family='iOS', version=(10, 3, 1), version_string='10.3.1')
Device(family='iPhone', brand='Apple', model='iPhone')


In [73]:
type(parse(user_agent[0]))

user_agents.parsers.UserAgent

In [74]:
data_test=df_groupby2.user_agent[:3]
print(data_test)
type(data_test)

id  num_sess
1   1           Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
    2           Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
    3           Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...
Name: user_agent, dtype: object


pandas.core.series.Series

- ** Parse data:**

In [75]:
%%time
#user_agent_parse=list(map(lambda x: parse(user_agent[x]), range(0,2344636)))
# tqdm.pandas(tqdm())
# tqdmuser_agent_parse = df_groupby2['user_agent'].progress_apply(lambda x: parse(x))

user_agent_parse = df_groupby2['user_agent'].apply(lambda x: parse(x))
print('user_agent_parse finished')

# data_browser = user_agent_parse.apply(lambda x: x.browser)
data_browser_family = user_agent_parse.apply(lambda x: x.browser.family)
data_browser_version_string = user_agent_parse.apply(lambda x: x.browser.version_string)
print('data_browser finished')

# data_os = user_agent_parse.apply(lambda x: x.os)
data_os_family = user_agent_parse.apply(lambda x: x.os.family)
data_os_version_string = user_agent_parse.apply(lambda x: x.os.version_string)
print('data_os finished')

# data_device = user_agent_parse.apply(lambda x: x.device)
data_device_family = user_agent_parse.apply(lambda x: x.device.family)
data_device_brand = user_agent_parse.apply(lambda x: x.device.brand)
data_device_model = user_agent_parse.apply(lambda x: x.device.model)
print('data_device finished')

user_agent_parse finished
data_browser finished
data_os finished
data_device finished
CPU times: user 1min 31s, sys: 134 ms, total: 1min 32s
Wall time: 1min 32s


- **Save Parsed data to csv:**

In [76]:
%%time
user_agent_parse.to_csv('../Data/Feature_Engineering/user_agent_parse.csv')

# data_browser.to_csv('../Data/data_browser.csv')
data_browser_family.to_csv('../Data/Feature_Engineering/data_browser_family.csv')
data_browser_version_string.to_csv('../Data/Feature_Engineering/data_browser_version_string.csv')

# data_os.to_csv('../Data/data_os.csv')
data_os_family.to_csv('../Data/Feature_Engineering/data_os_family.csv')
data_os_version_string.to_csv('../Data/Feature_Engineering/data_os_version_string.csv')

# data_device.to_csv('../Data/data_device.csv')
data_device_family.to_csv('../Data/Feature_Engineering/data_device_family.csv')
data_device_model.to_csv('../Data/Feature_Engineering/data_device_model.csv')
data_device_brand.to_csv('../Data/Feature_Engineering/data_device_brand.csv')
# Read saved csv:
# pd.read_csv('../Data/user_agent_parse.csv')

CPU times: user 4.29 s, sys: 95 ms, total: 4.38 s
Wall time: 4.4 s


- **Read user_agent data:**

In [77]:
%%time
# must add:
# ,header=None
# after the path to avoid making the first line become column names
user_agent=pd.read_csv('../Data/Feature_Engineering/user_agent_parse.csv',header=None)

data_browser_family=pd.read_csv('../Data/Feature_Engineering/data_browser_family.csv',header=None)
data_browser_version_string=pd.read_csv('../Data/Feature_Engineering/data_browser_version_string.csv',header=None)

data_os_family=pd.read_csv('../Data/Feature_Engineering/data_os_family.csv',header=None)
data_os_version_string=pd.read_csv('../Data/Feature_Engineering/data_os_version_string.csv',header=None)

data_device_family=pd.read_csv('../Data/Feature_Engineering/data_device_family.csv',header=None)
data_device_model=pd.read_csv('../Data/Feature_Engineering/data_device_model.csv',header=None)
data_device_brand=pd.read_csv('../Data/Feature_Engineering/data_device_brand.csv',header=None)

CPU times: user 512 ms, sys: 7 ms, total: 519 ms
Wall time: 520 ms


In [81]:
df_groupby2[:3]

session_id                      finger_print  \
id num_sess                                                                 
1  1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
   2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
   3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   

                      date_time referrer  \
id num_sess                                
1  1        2017-05-19 08:02:53            
   2        2017-05-19 13:51:59            
   3        2017-06-02 21:40:06            

                                                      page_url  \
id num_sess                                                      
1  1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

            product_viewed_browse            cart_started  \
id num_sess                                                 
1  1                          NaT 2017-05-19 08:02:51.292   
   2                          NaT 2017-05-19 08:02:51.292   
   3                          NaT 2017-06-02 21:40:06.147   

            basket_paid_browsing  \
id num_sess                        
1  1                        None   
   2                        None   
   3                        None   

                                                 product_name product_save  \
id num_sess                                                                  
1  1                                                     None         None   
   2                                                     None         None   
   3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
1  1                 None              None   nun      None  1495180971292   
   2                 None              None   nun      None  1495180971292   
   3                    9       03:01:20:04   nun      None  1496439606147   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
1  1                       None  None            False         None   
   2                       None  None            False         None   
   3                       None     9            False         None   

            email_click_through       cart_firstTime        cart_lastTime  \
id num_sess                                                                 
1  1                       True  2017-05-19 09:02:53  2017-05-19 09:02:53   
   2                       True  2017-05-19 14:51:59  2017-05-19 14:51:59   
   3                      False  2017-06-02 22:40:06  2017-06-02 22:40:06   

            basketInfoTime      ip_address  \
id num_sess                                  
1  1            1495182843  77.103.157.226   
   2            1495203782  77.103.157.226   
   3            1496441524  77.103.157.226   

                                                    user_agent  city country  \
id num_sess                                                                    
1  1         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   2         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   3         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   

             dwell_time  weekday  days  month time_of_day  cart_time  \
id num_sess                                                            
1  1           01:00:00        4    19      5    08:02:53     0 days   
   2           01:00:00        4    19      5    13:51:59     0 days   
   3           01:00:00        4     2      6    21:40:06     0 days   

             num_browsing  
id num_sess                
1  1                    1  
   

In [82]:
print(df_groupby2.shape)
print(type(df_groupby2))
print(data_browser_family.shape)
print(type(data_browser_family))

(220047, 34)
<class 'pandas.core.frame.DataFrame'>
(220047, 3)
<class 'pandas.core.frame.DataFrame'>


In [90]:
data_browser_family.loc[:4,2]

0    Mobile Safari
1    Mobile Safari
2    Mobile Safari
3    Mobile Safari
4           Chrome
Name: 2, dtype: object

In [91]:
# add extra columns to data table
df_groupby2['data_browser_family']=data_browser_family.loc[:,2].values
df_groupby2['data_browser_version_string']=data_browser_version_string.loc[:,2].values

df_groupby2['data_os_family']=data_os_family.loc[:,2].values
df_groupby2['data_os_version_string']=data_os_version_string.loc[:,2].values

df_groupby2['data_device_family']=data_device_family.loc[:,2].values
df_groupby2['data_device_model']=data_device_model.loc[:,2].values
df_groupby2['data_device_brand']=data_device_brand.loc[:,2].values

**Save df_groupby2**

In [93]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

CPU times: user 1.75 s, sys: 491 ms, total: 2.25 s
Wall time: 3.43 s


In [92]:
df_groupby2[:3]

session_id                      finger_print  \
id num_sess                                                                 
1  1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
   2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
   3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   

                      date_time referrer  \
id num_sess                                
1  1        2017-05-19 08:02:53            
   2        2017-05-19 13:51:59            
   3        2017-06-02 21:40:06            

                                                      page_url  \
id num_sess                                                      
1  1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
   3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   

            product_viewed_browse            cart_started  \
id num_sess                                                 
1  1                          NaT 2017-05-19 08:02:51.292   
   2                          NaT 2017-05-19 08:02:51.292   
   3                          NaT 2017-06-02 21:40:06.147   

            basket_paid_browsing  \
id num_sess                        
1  1                        None   
   2                        None   
   3                        None   

                                                 product_name product_save  \
id num_sess                                                                  
1  1                                                     None         None   
   2                                                     None         None   
   3         Thermal Blackout Curtains - 9 Colours & 5 Sizes!          75%   

            product_price product_time_left email Post_Code cart_startTime  \
id num_sess                                                                  
1  1                 None              None   nun      None  1495180971292   
   2                 None              None   nun      None  1495180971292   
   3                    9       03:01:20:04   nun      None  1496439606147   

            product_viewed_cart total basket_paid_cart payment_type  \
id num_sess                                                           
1  1                       None  None            False         None   
   2                       None  None            False         None   
   3                       None     9            False         None   

            email_click_through       cart_firstTime        cart_lastTime  \
id num_sess                                                                 
1  1                       True  2017-05-19 09:02:53  2017-05-19 09:02:53   
   2                       True  2017-05-19 14:51:59  2017-05-19 14:51:59   
   3                      False  2017-06-02 22:40:06  2017-06-02 22:40:06   

            basketInfoTime      ip_address  \
id num_sess                                  
1  1            1495182843  77.103.157.226   
   2            1495203782  77.103.157.226   
   3            1496441524  77.103.157.226   

                                                    user_agent  city country  \
id num_sess                                                                    
1  1         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   2         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   
   3         Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...  None      GB   

             dwell_time  weekday  days  month time_of_day  cart_time  \
id num_sess                                                            
1  1           01:00:00        4    19      5    08:02:53     0 days   
   2           01:00:00        4    19      5    13:51:59     0 days   
   3           01:00:00        4     2      6    21:40:06     0 days   

             num_browsing data_browser_family data_browser_version_string  \
id num_ses

---
## 10. Referrer
Here, we only check if the first referrer is empty

**Load df_groupby2:**

In [106]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2=pickle.load(f)

CPU times: user 1.05 s, sys: 147 ms, total: 1.2 s
Wall time: 1.94 s


In [94]:
df_groupby2.referrer.iloc[1]

''

In [95]:
def referrer_check(referrer):  
    if referrer == '':
        return 0
    else: 
        return 1

In [96]:
df_groupby2['referrer_binary']=df_groupby2.referrer.progress_apply(lambda x: referrer_check(x))

100%|██████████| 220047/220047 [00:00<00:00, 686235.87it/s]


In [101]:
df_groupby2.loc[[52432,52433],:]

session_id                      finger_print  \
id    num_sess                                                                 
52432 1           4hsrbvc10smrwecmxjz2u3xx  5645078a67cde02aea81f06f54e9728e   
      2           u8y8pav7275qjq83zfm6tlk8  5645078a67cde02aea81f06f54e9728e   
      3           alqqfekxqvedprr1jdf8zqpv  5645078a67cde02aea81f06f54e9728e   
      4         1waht5m3hhnwa1pvz00evaj2ac  5645078a67cde02aea81f06f54e9728e   
52433 1           zxkbfysuold2vl2v4el3ubzv  17e96f528f8940c9392619c58b8b136d   
      2           vxhv5kka1iu93rtmz4g6osc4  de6f52700a2fefcd15e803abbe23cde1   
      3          15ox967rdf7q3yjn65eedevsh  de6f52700a2fefcd15e803abbe23cde1   
      4          1tz5t8tsg2bdj5c3w6i1r5ufu  3a131efaa20b6a197508313f0eb266c4   

                         date_time  \
id    num_sess                       
52432 1        2017-05-20 18:20:53   
      2        2017-05-20 18:27:57   
      3        2017-05-20 18:38:25   
      4        2017-05-20 20:26:03   
52433 1        2017-05-24 18:48:37   
      2        2017-06-12 14:03:52   
      3        2017-06-12 14:03:52   
      4        2017-06-12 14:15:41   

                                                         referrer  \
id    num_sess                                                      
52432 1                 android-app%3A%2F%2Fcom.google.android.gm   
      2         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      3         https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...   
      4                 android-app%3A%2F%2Fcom.google.android.gm   
52433 1         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      2         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      3         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      4                                                             

                                                         page_url  \
id    num_sess                                                      
52432 1         https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...   
      2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      3         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      4         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
52433 1         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      2         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      3         https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...   
      4          https%3A%2F%2Fwww.wowcher.co.uk%2Fdeals%2Flondon   

                 product_viewed_browse            cart_started  \
id    num_sess                                                   
52432 1        2017-05-20 18:20:52.984                     NaT   
      2                            NaT 2017-05-20 18:27:56.970   
      3                            NaT 2017-05-20 18:38:24.699   
      4                            NaT 2017-05-20 20:26:01.808   
52433 1                            NaT 2017-05-24 18:48:36.464   
      2                            NaT 2017-06-12 14:03:52.170   
      3                            NaT 2017-06-12 14:03:52.155   
      4                            NaT                     NaT   

               basket_paid_browsing  \
id    num_sess                        
52432 1                        None   
      2                        None   
      3                        None   
      4                        None   
52433 1                        None   
      2                        None   
      3                        None   
      4                        None   

                                                     product_name  \
id    num_sess                                                      
52432 1                            Teepee Tent - Sleeps 6 People!   
      2         6 Sessions of IPL Hair Removal @ Naturalaser -...   
      3         6 Sessions of IPL Hair Removal @ Naturalaser -...   
      4         6 Sessions of IPL Hair Removal @ Naturalaser -...   
52

**Save df_groupby2:**

In [102]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "wb" ) as f:
    pickle.dump(df_groupby2, f)

CPU times: user 2.04 s, sys: 424 ms, total: 2.46 s
Wall time: 3.66 s


---
# Feature Engineering Part 2:
**Convert Features into proper form that is suitable to feed into models**

**Load df_groupby2:**

In [4]:
%%time
with open( "../Data/Feature_Engineering/df_groupby2.p", "rb" ) as f:
    df_groupby2=pickle.load(f)

CPU times: user 936 ms, sys: 378 ms, total: 1.31 s
Wall time: 1.94 s


## Core dataframe: df_features

In [5]:
df_features=df_groupby2.reset_index()

---
## 1. Time Features Re-engineering
**Convert all time features to abosolute time:**

---

**Define a absolute time conversion function:**


In [140]:
# timestamp function to 
def absolute_time(datetime):
    return time.mktime(datetime.timetuple())

### 1.1 date_time

In [141]:
print(type(df_features['date_time'][0]))
df_features['date_time'][0]

<class 'pandas.tslib.Timestamp'>


Timestamp('2017-05-19 08:02:53')

In [142]:
df_features['date_time'] = df_features.date_time.apply(lambda x: absolute_time(x) if not pd.isnull(x) else 0).values

In [143]:
df_features['date_time'][:3]

0    1.495181e+09
1    1.495202e+09
2    1.496440e+09
Name: date_time, dtype: float64

**Save df_features**

In [144]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "wb" ) as f:
    pickle.dump(df_features, f)

CPU times: user 1.83 s, sys: 390 ms, total: 2.22 s
Wall time: 2.22 s


### 1.2 time_to_cart
**It measures the time from first browsing to first time add items to cart**
<br> **time_to_cart = cart_firstTime - first_browsing_time**

**Load df_features**

In [174]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "rb" ) as f:
    df_features=pickle.load(f)

CPU times: user 931 ms, sys: 340 ms, total: 1.27 s
Wall time: 1.27 s


**Obtain cart_firstTime:**

In [175]:
# cart_firstTime is same for same session
print(type(df_features.cart_firstTime[0]))
print(df_features.cart_firstTime[0])
df_features.cart_firstTime.astype('datetime64[ns]')

df_features.cart_firstTime=df_features.cart_firstTime.astype('datetime64[ns]')
print(type(df_features.cart_firstTime[0]))

<class 'str'>
2017-05-19 09:02:53
<class 'pandas.tslib.Timestamp'>


In [176]:
df_features['cart_firstTime'] = df_features.cart_firstTime.apply(lambda x: absolute_time(x) if not pd.isnull(x) else 0).values

In [177]:
df_features['cart_firstTime'][:3]

0    1.495185e+09
1    1.495206e+09
2    1.496443e+09
Name: cart_firstTime, dtype: float64

**Obtain time_to_cart:**

In [178]:
df_features['time_to_cart'] = df_features['cart_firstTime']-df_features['date_time']

In [179]:
# assign negative values to be 0
df_features.time_to_cart[df_features.time_to_cart<0]=0

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


**Save df_features**

In [180]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "wb" ) as f:
    pickle.dump(df_features, f)

CPU times: user 1.83 s, sys: 445 ms, total: 2.28 s
Wall time: 3.48 s


### 1.3 Product_time_left

**Load df_features**

In [ ]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "rb" ) as f:
    df_features=pickle.load(f)

In [183]:
df_features.product_time_left[:5]

0           None
1           None
2    03:01:20:04
3    00:08:36:54
4    06:09:34:40
Name: product_time_left, dtype: object

In [184]:
# "%d:%H:%M:%S"
print(type(df_features['product_time_left'][0]))
df_features['product_time_left'][0]

<class 'NoneType'>


In [185]:
def product_time_left(timeleft):
    if not timeleft == None:
        try:
            if(timeleft[timeleft.rfind(':')+1:]!='60'):
                t = datetime.datetime.strptime(timeleft, "%d:%H:%M:%S")
                delta = datetime.timedelta(days=t.day, hours=t.hour, minutes=t.minute, seconds=t.second)
                return delta.total_seconds()
            else:
                timeleft = timeleft[:timeleft.rfind(':')+1]+"00"
                t = datetime.datetime.strptime(timeleft, "%d:%H:%M:%S")
                delta = datetime.timedelta(days=t.day, hours=t.hour, minutes=t.minute, seconds=t.second)
                return delta.total_seconds()
        except ValueError:
            if ':' in timeleft and 'NaN' not in timeleft: #filter some strange uncleaned terms out
                timeleft = timeleft[timeleft.index(':')+1:]
                t = datetime.datetime.strptime(timeleft, "%H:%M:%S")
                delta = datetime.timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
                return delta.total_seconds()
            else:
                return 0
    else:
        return 0

In [186]:
%%time
df_features.product_time_left = df_features['product_time_left'].apply(lambda x: product_time_left(x))

CPU times: user 2.91 s, sys: 25 ms, total: 2.94 s
Wall time: 2.94 s


In [187]:
df_features.product_time_left[:5]

0         0.0
1         0.0
2    264004.0
3     31014.0
4    552880.0
Name: product_time_left, dtype: float64

**Save df_features**

In [189]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "wb" ) as f:
    pickle.dump(df_features, f)

CPU times: user 1.78 s, sys: 387 ms, total: 2.17 s
Wall time: 3.35 s


### 1.4 Time_to_product_viewed
**Measure how long does it takes for customers to view product**
<br>**(product_viewed_time - first browsing time)**

---
**Warning: Don't Run this block again!!!!!!**
---
**import df:**

In [211]:
%%time
with open( "../Data/data_final.p", "rb" )  as f:
    df=pickle.load(f)

CPU times: user 5.06 s, sys: 1.98 s, total: 7.04 s
Wall time: 11.8 s


**Check data format:**

In [212]:
print('product_viewed_browse:\n',df['product_viewed_browse'][[0,1,8,9]])
print('\nproduct_viewed_cart:\n',df['product_viewed_cart'][[0,12]])

product_viewed_browse:
 0                       NaT
1                       NaT
8   2017-06-12 08:32:43.033
9                       NaT
Name: product_viewed_browse, dtype: datetime64[ns]

product_viewed_cart:
 0              None
12    1497448247032
Name: product_viewed_cart, dtype: object


**product_viewed_browse (to float):**
<br>actually using integer would be more convenient, but here using float for better display

In [213]:
%%time
# transform data into form of total second
df['product_viewed_browse'] = df.product_viewed_browse.apply(lambda x: absolute_time(x) if not pd.isnull(x) else 0).values
print('product_viewed_browse:\n',df['product_viewed_browse'][[0,8,9]])

product_viewed_browse:
 0    0.000000e+00
8    1.497256e+09
9    0.000000e+00
Name: product_viewed_browse, dtype: float64
CPU times: user 5.58 s, sys: 631 ms, total: 6.21 s
Wall time: 6.52 s


**product_viewed_cart:**

In [214]:
%%time
# transform data to type 'float'
df['product_viewed_cart']=df['product_viewed_cart'].astype('float').copy()
# replace NaN term to 0
df['product_viewed_cart'][(np.isnan(df['product_viewed_cart']).copy())] = 0
print('product_viewed_cart\n',df['product_viewed_cart'].loc[[0,12]])

product_viewed_cart
 0     0.000000e+00
12    1.497448e+12
Name: product_viewed_cart, dtype: float64
CPU times: user 1.07 s, sys: 176 ms, total: 1.24 s
Wall time: 1.24 s


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


**delete final 3 digit of product_viewed_cart**

In [367]:
def delete_3_digit(x):
    return np.float(math.trunc(x/1000))

df['product_viewed_cart']=df['product_viewed_cart'].apply(lambda x: delete_3_digit(x))

In [368]:
print(df['product_viewed_cart'][[0,12]])

0     0.000000e+00
12    1.497448e+09
Name: product_viewed_cart, dtype: float64


**Unite the common info of 2 columns to get 'product_viewed_time'**

In [369]:
df['product_viewed_time'] = df['product_viewed_cart'].copy
df['product_viewed_time'][df['product_viewed_cart']==0]=df['product_viewed_browse']

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [371]:
sum(df['product_viewed_time']!=0)

1015096


** Save df:**

In [376]:
%%time
with open( "../Data/data_final.p", "wb" ) as f:
    pickle.dump(df, f)

CPU times: user 11.2 s, sys: 3.07 s, total: 14.3 s
Wall time: 21.1 s


---

In [54]:
# import df:
with open( "../Data/data_final.p", "rb" )  as f:
    df=pickle.load(f)

In [26]:
df.sort_values(['id','num_sess','product_viewed_time'],inplace=True,ascending=[True, True, True])

In [79]:
df['product_viewed_time'].loc[df['product_viewed_time',:]==0]='nan'

TypeError: unhashable type: 'slice'

In [63]:
a = df
# a=df.copy()
a.loc[8,'session_id']='nan'
a

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country,product_viewed_time
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,0.000000e+00,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,0.000000e+00,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,0.000000e+00,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,0.000000e+00,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,0.000000e+00,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,0.000000e+00,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,0.000000e+00,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,0.000000e+00,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
4,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,1.495805e+09,NaT,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,0.000000e+00,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,1.495805e+09
5,2,1,nan,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,0.000000e+00,2017-05-26 13:25:20.891,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,0.000000e+00,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,0.000000e+00
6,2,2,nan,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:22,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,1.497014e+09,NaT,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,None,None,1497014430749,0.000000e+00,229,False,None,False,2017-06-09 14:20:23,2017-06-09 14:20:23,1497016328,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,1.497014e+09
7,2,2,nan,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:23,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,0.000000e+00,2017-06-09 13:20:30.749,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,None,None,1497014430749,0.000000e+00,229,False,None,False,2017-06-09 14:20:23,2017-06-09 14:20:23,1497016328,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,0.000000e+00
8,2,3,nan,e5f3ca057208f51a2cacae58ff95d757,2017-06-12 08:32:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,1.497256e+09,NaT,None,Modern Shaggy Rug - 10 Colours & 6 Sizes!,64%,9,03:14:27:15,None,None,1497256364377,0.000000e+00,9,False,None,False,2017-06-12 09:32:26,2017-06-12 09:32:26,149

In [42]:
df_product_viewed_time = df.groupby(['id','num_sess'],sort=False).first()
df_product_viewed_time

session_id                      finger_print  \
id    num_sess                                                                 
1     1           k79c9lk1nfop9m4wx0fmrg16  5c6eaac3c9db786d83c28c4d673ff556   
      2            w2gunkrgoifz0j4agqb8bz4  5c6eaac3c9db786d83c28c4d673ff556   
      3         14qy38xtlrcnc1lamkqq6fqd84  5c6eaac3c9db786d83c28c4d673ff556   
      4         1vf4mz1acp57x1p1zlyz5xebe6  5c6eaac3c9db786d83c28c4d673ff556   
2     1          qvsw8ty43cbb15tu52ujywjs3  e5f3ca057208f51a2cacae58ff95d757   
      2           xe1kopsb1o573zhid3i51eds  e5f3ca057208f51a2cacae58ff95d757   
      3         184grb3zg9udt1scfxr2697n5b  e5f3ca057208f51a2cacae58ff95d757   
      4          11expdzyaf01l4rt3b953me14  e5f3ca057208f51a2cacae58ff95d757   
3     1         1fxh0wekr0wq81on76gwu04gap  c91effe8aa9659c7f41e1fcab45c7db6   
      2           tf3ruinsik6kmfdxx2bzy6mz  c91effe8aa9659c7f41e1fcab45c7db6   
      3         1wjziazqo6cjz1tzkyw11s34kw  c91effe8aa9659c7f41e1fcab45c7db6   
4     1           c2m4om7rtcuobvhdyebqldtt  ace3271b5566133d8f5293ef3ad838f4   
      2           dazqn52newe1htllvdhvfwv2  ace3271b5566133d8f5293ef3ad838f4   
      3          24b0q2u0kl4u1k4asjg42nyww  ace3271b5566133d8f5293ef3ad838f4   
5     1          ohtumc6e1a161tlzljavs6ysg  4d92df9e764d0843b284916dbbd177d2   
      2         1gzv3pih1tocs1reyrqmslitug  4d92df9e764d0843b284916dbbd177d2   
      3          56kf701hy2ql1vzm1gqa2bxfk  4d92df9e764d0843b284916dbbd177d2   
6     1           81xrd850s3unnvgbaliaubvd  15e47ddc6e4a60a0ca4c8ffd398da802   
      2          re6zyhp865ct1n2943as7fxzn  15e47ddc6e4a60a0ca4c8ffd398da802   
      3          1c1myrju2fh1xm6a6b7qq7a4t  15e47ddc6e4a60a0ca4c8ffd398da802   
7     1           jxk9vj7imafmhkin1ecgy1e4  9437fefd1fa64447d8fa5e2b47d41ebb   
      2          mc51p1g18vle1xaavzof357jk  9437fefd1fa64447d8fa5e2b47d41ebb   
      3          c7zggsemrcnd18m2600ljriq8  9437fefd1fa64447d8fa5e2b47d41ebb   
8     1         142x000za7h8t1ghszaxqjln57  96b3b651096e1de1de42a89c7fc52a15   
      2           6b31hw6kqd41plav6th846ca  96b3b651096e1de1de42a89c7fc52a15   
      3          70xwmk77rxuk1x1oovb20g2as  96b3b651096e1de1de42a89c7fc52a15   
      4           mmex3jtt7b2484okrg77ien3  96b3b651096e1de1de42a89c7fc52a15   
      5          p6se6ocfjh971nsm17hllspc6  96b3b651096e1de1de42a89c7fc52a15   
9     1         13hw7mfpre9h81cokg67boum8a  7b005ee5adabb3602ab3b6a611fce1e9   
      2         1hyrkh4zppz221wkdbsjcs9qtj  7b005ee5adabb3602ab3b6a611fce1e9   
...                                    ...                               ...   
52425 4         1i4ydv5u8g7qr1r1axiyzrsb19  d4dbc4053874d0c00490daed3e55ce27   
      5           khhegte59nw0ihcx16smrh17  d4dbc4053874d0c00490daed3e55ce27   
52426 1         1fjr9dhvndgk01u8fqeeeewi4q  070c275ad724f7d5cad9ccb51164d11c   
      2         1j96usak1al4n19flhmfapi7vs  f68874ae4e14acab3ac909f8762c9ad1   
      3         1gkwtryaj59ka13i8byokpv17r  f68874ae4e14acab3ac909f8762c9ad1   
      4           8bcifp1ll0fukdzen0eqyv1b  070c275ad724f7d5cad9ccb51164d11c   
      5         10lzrhikr66bh119a9psrye73d  070c275ad724f7d5cad9ccb51164d11c   
52427 1          10nd96wftg912fqav23zfdshi  c4ccba8f95a27f43c8b26e2621647b40   
      2          11cic0ykuzumziqnzk6wf4bj1  c4ccba8f95a27f43c8b26e2621647b40   
      3           3u4lf9io48s574q6chrf98ai  c4ccba8f95a27f43c8b26e2621647b40   
52428 1         1gdcbph15uhwu1l1odbte3w3st  8fa7f3b3e742b0a70e7f7a6fbe878acb   
      2          1rl8gt4ltvzgkrhc95mp3fdyj  8fa7f3b3e742b0a70e7f7a6fbe878acb   
      3          19h6ulg6yk7a8xsl3eotd4p9r  8fa7f3b3e742b0a70e7f7a6fbe878acb   
52429 1          1uiefg1sr9e23zidbe2n8yvkx  0c00dc02f3f5a9614039489510f42dc6   
      2         1pebg0xxevswx10jjruieicrsc  070c275ad724f7d5cad9ccb51164d11c   
      3          1g8rxaa8saxm1wcydwy4p59r7  3137c3cec93a34b8760c8248c910de7e   
52430 1         16o2tg7fi6xhg1gixi31bume6s  0a98458c4c3862a2167142f22b9eff19   
      2          g9hgpmlsq1es1bh

In [34]:
df

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country,product_viewed_time
0,1,1,k79c9lk1nfop9m4wx0fmrg16,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 08:02:53,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,0.000000e+00,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,0.000000e+00,None,False,None,True,2017-05-19 09:02:53,2017-05-19 09:02:53,1495182843,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
1,1,2,w2gunkrgoifz0j4agqb8bz4,5c6eaac3c9db786d83c28c4d673ff556,2017-05-19 13:51:59,19,,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,0.000000e+00,2017-05-19 08:02:51.292,None,None,None,None,None,None,None,1495180971292,0.000000e+00,None,False,None,True,2017-05-19 14:51:59,2017-05-19 14:51:59,1495203782,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
2,1,3,14qy38xtlrcnc1lamkqq6fqd84,5c6eaac3c9db786d83c28c4d673ff556,2017-06-02 21:40:06,2,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,0.000000e+00,2017-06-02 21:40:06.147,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,03:01:20:04,None,None,1496439606147,0.000000e+00,9,False,None,False,2017-06-02 22:40:06,2017-06-02 22:40:06,1496441524,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
3,1,4,1vf4mz1acp57x1p1zlyz5xebe6,5c6eaac3c9db786d83c28c4d673ff556,2017-06-09 14:23:10,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,0.000000e+00,2017-06-09 14:23:10.017,None,Thermal Blackout Curtains - 9 Colours & 5 Sizes!,75%,9,00:08:36:54,None,None,1497018190017,0.000000e+00,9,False,None,False,2017-06-09 15:23:10,2017-06-09 15:23:10,1497020106,77.103.157.226,Mozilla/5.0+(iPhone;+CPU+iPhone+OS+10_3_1+like...,None,GB,0.000000e+00
5,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,0.000000e+00,2017-05-26 13:25:20.891,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,0.000000e+00,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,0.000000e+00
4,2,1,qvsw8ty43cbb15tu52ujywjs3,e5f3ca057208f51a2cacae58ff95d757,2017-05-26 13:25:27,26,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,1.495805e+09,NaT,None,3 5 or 7nt Algarve Break & Flights,26%,69,06:09:34:40,None,None,1495805120891,0.000000e+00,69,False,None,False,2017-05-26 14:25:27,2017-05-26 14:25:27,1495807024,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,1.495805e+09
7,2,2,xe1kopsb1o573zhid3i51eds,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:23,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,0.000000e+00,2017-06-09 13:20:30.749,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,None,None,1497014430749,0.000000e+00,229,False,None,False,2017-06-09 14:20:23,2017-06-09 14:20:23,1497016328,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,0.000000e+00
6,2,2,xe1kopsb1o573zhid3i51eds,e5f3ca057208f51a2cacae58ff95d757,2017-06-09 13:20:22,9,,https%3A%2F%2Fwww.wowcher.co.uk%2Femail-deals%...,1.497014e+09,NaT,None,7nt 4* All-Inclusive Croatia & Flights - Summe...,27%,229,04:09:39:29,None,None,1497014430749,0.000000e+00,229,False,None,False,2017-06-09 14:20:23,2017-06-09 14:20:23,1497016328,148.252.216.66,Mozilla/5.0+(Windows+NT+6.0;+WOW64)+AppleWebKi...,None,GB,1.497014e+09
9,2,3,184grb3zg9udt1scfxr2697n5b,e5f3ca057208f51a2cacae58ff95d757,2017-06-12 08:32:26,12,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,0.000000e+00,2017-06-12 08:32:44.377,None,Modern Shaggy Rug - 10 Colours & 6 Sizes!,64%,9,03:1

In [20]:
sum(df_product_viewed_time['product_viewed_time']!=0)

95914

In [407]:
df_product_viewed_time['product_viewed_time'].shape

(220047,)

In [409]:
sum(df_product_viewed_time['product_viewed_time']!=0)

87439

In [405]:
df_features['date_time'].shape

(220047,)

In [408]:
df_product_viewed_time['product_viewed_time']-df_features['date_time']

-1495180973.0

### 1.5 Dwell time

In [213]:
print(type(df_features['dwell_time'][0]))
print(df_features['dwell_time'][0])
print(df_features['dwell_time'][0].total_seconds())

df_features['dwell_time']=df_features['dwell_time'].apply(lambda x: x.total_seconds() if not pd.isnull(x) else 0).values

<class 'pandas.tslib.Timedelta'>
0 days 01:00:00
3600.0


In [214]:
df_features['dwell_time'][:3]

0    3600.0
1    3600.0
2    3600.0
Name: dwell_time, dtype: float64

### 1.6 time_of_day

In [215]:
print(type(df_features['time_of_day'][0]))
print(df_features['time_of_day'][0])

<class 'datetime.time'>
08:02:53


In [216]:
def hms_to_seconds(t):
    h, m, s = [int(i) for i in t.split(':')]
    return 3600*h + 60*m + s

In [217]:
df_features['time_of_day']=df_features['time_of_day'].apply(lambda x: hms_to_seconds(str(x)))

In [218]:
df_features['time_of_day'][:3]

0    28973
1    49919
2    78006
Name: time_of_day, dtype: int64

### 1.7 cart_time

In [219]:
print(type(df_features['cart_time'][0]))
print(df_features['cart_time'][0])
print(df_features['cart_time'][0].total_seconds())

<class 'pandas.tslib.Timedelta'>
0 days 00:00:00
0.0


In [220]:
df_features['cart_time'] = df_features['cart_time'].apply(lambda x: x.total_seconds())

In [221]:
df_features['cart_time'][:8]

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7    67.0
Name: cart_time, dtype: float64

### Show df_features

In [465]:
df_features

,num_sess,basketPaid,cart_firstTime,city,country,date_time,email,email_click_through,payment_type,product_price,product_save,product_time_left,product_viewed,total,dwell_time,weekday,days,month,time_of_day,cart_time,num_browsing,data_browser_family,data_browser_version_string,data_os_family,data_os_version_string,data_device_family,data_device_model,data_device_brand,referrer_binary
0,1,False,1.496607e+09,None,GB,1.496603e+09,gmail,False,None,69.00,86%,13999.0,1.496603e+09,69.00,3612.0,6,4,6,68800,11.0,4,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
1,2,False,1.496915e+09,None,GB,1.496911e+09,gmail,False,None,49.00,51%,224461.0,0.000000e+00,49.00,3709.0,3,8,6,31031,1.0,15,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
2,3,False,1.495467e+09,None,GB,1.495464e+09,gmail,False,None,99.00,51%,289635.0,0.000000e+00,99.00,3625.0,0,22,5,52338,18.0,7,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
3,4,False,1.497270e+09,None,GB,1.497266e+09,gmail,False,None,19.00,80%,301443.0,0.000000e+00,19.00,3605.0,0,12,6,40553,0.0,4,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
4,5,False,1.497180e+09,None,GB,1.497177e+09,gmail,False,None,10.00,96%,0.0,0.000000e+00,10.00,3601.0,6,11,6,37613,1.0,2,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,1
5,6,False,1.496085e+09,Belfast,GB,1.496081e+09,gmail,False,None,199.00,49%,277081.0,0.000000e+00,199.00,3645.0,0,29,5,64896,21.0,7,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
6,7,False,1.495235e+09,London,GB,1.495231e+09,gmail,False,paypalExpress,159.00,50%,261999.0,0.000000e+00,159.00,4311.0,4,19,5,79301,685.0,24,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
7,8,False,1.497006e+09,None,GB,1.497004e+09,gmail,False,None,79.00,49%,304736.0,0.000000e+00,79.00,3600.0,4,9,6,37293,1000.0,1,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,1
8,9,False,1.496823e+09,None,GB,1.496821e+09,gmail,False,None,79.00,51%,314240.0,0.000000e+00,79.00,4127.0,2,7,6,27232,1765.0,168,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
9,10,False,1.496482e+09,None,GB,1.496480e+09,gmail,False,None,29.00,68%,310297.0,1.496480e+09,29.00,3766.0,5,3,6,31537,1622.0,4,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0


### Save df_features

In [466]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "wb" ) as f:
    pickle.dump(df_features, f)

CPU times: user 601 ms, sys: 156 ms, total: 757 ms
Wall time: 984 ms


---
## 2. Categories Features Re-engineering

### Load df_features

In [223]:
%%time
with open( "../Data/Feature_Engineering/df_features.p", "rb" ) as f:
    df_features=pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Feature_Engineering/df_features.p'

---

In [224]:
features_category=['num_sess','city','country','email','email_click_through','payment_type','product_price',
                   'product_save','total','num_browsing','data_browser_family','data_os_family','data_device_brand',
                   'referrer_binary']

In [225]:
df_features[features_category].columns

Index(['num_sess', 'city', 'country', 'email', 'email_click_through',
       'payment_type', 'product_price', 'product_save', 'total',
       'num_browsing', 'data_browser_family', 'data_os_family',
       'data_device_brand', 'referrer_binary'],
      dtype='object')

In [226]:
df_features[features_category][:3]

,num_sess,city,country,email,email_click_through,payment_type,product_price,product_save,total,num_browsing,data_browser_family,data_os_family,data_device_brand,referrer_binary
0,1,None,GB,nun,True,None,None,None,None,1,Mobile Safari,iOS,Apple,0
1,2,None,GB,nun,True,None,None,None,None,1,Mobile Safari,iOS,Apple,0
2,3,None,GB,nun,False,None,9,75%,9,1,Mobile Safari,iOS,Apple,0


### 2.1 City

In [227]:
df_features.city.value_counts()[:50]

London                 20254
Manchester              4357
Birmingham              2405
Derby                   2393
Liverpool               2024
Nottingham              2006
Glasgow                 1977
Leeds                   1738
Bristol                 1594
Edinburgh               1409
Sheffield               1296
El Segundo              1234
Leicester               1171
Coventry                 887
Cardiff                  805
Southampton              805
Northampton              734
Milton Keynes            719
Reading                  707
Newcastle Upon Tyne      698
Bolton                   664
Stoke-on-trent           653
Plymouth                 629
Oxford                   611
Stockport                572
Preston                  568
Brighton                 546
Belfast                  542
Croydon                  534
Wolverhampton            534
Hull                     521
Swindon                  511
Cambridge                506
Bradford                 492
Gloucester    

In [228]:
city=df_features.city.value_counts().index

In [229]:
city

Index(['London', 'Manchester', 'Birmingham', 'Derby', 'Liverpool',
       'Nottingham', 'Glasgow', 'Leeds', 'Bristol', 'Edinburgh',
       ...
       'Haltwhistle', 'Kaunas', 'Lochwinnoch', 'Thornhaugh', 'Toledo',
       'Llandeilo', 'Alva', 'Priors Marston', 'Abertillery', 'Langport'],
      dtype='object', length=2242)

In [230]:
'London' in city

True

In [231]:
def city_class(x):
  if x in city:
    return x
  else:
    return 'Others'

In [232]:
df_features['city']=df_features['city'].apply(lambda x: city_class(x))

In [233]:
df_features['city'][:6]

0    Others
1    Others
2    Others
3    Others
4    Others
5    Others
Name: city, dtype: object

### 2.2 Country

In [234]:
df_features.country.value_counts()[:15]

GB    212148
US      2840
IN      1105
DE       476
ES       378
CH       365
IE       332
LK       213
EU       189
FR       171
NL       153
IM       117
GR       100
RO        83
CY        82
Name: country, dtype: int64

In [235]:
country=df_features.country.value_counts()[:15].index

In [236]:
country

Index(['GB', 'US', 'IN', 'DE', 'ES', 'CH', 'IE', 'LK', 'EU', 'FR', 'NL', 'IM',
       'GR', 'RO', 'CY'],
      dtype='object')

In [237]:
def country_class(x):
  if x in country:
    return x
  else:
    return 'Others'

In [238]:
df_features['country']=df_features['country'].apply(lambda x: country_class(x))

In [239]:
df_features['country'][:3]

0    GB
1    GB
2    GB
Name: country, dtype: object

### 2.3 Email

In [240]:
df_features.email.value_counts()[:30]

nun            57394
hotmail        54724
gmail          31506
yahoo          20280
live            6771
aol             5622
btinternet      5534
outlook         5075
sky             5008
icloud          4515
googlemail      3048
msn             1844
ntlworld        1517
talktalk        1465
virginmedia      898
me               791
blueyonder       730
ymail            728
tiscali          695
mail             458
rocketmail       392
wowcher          338
btopenworld      287
gmx              209
talk21           159
wp               150
btconnect        149
virgin           149
nhs              138
tesco            134
Name: email, dtype: int64

In [241]:
email=df_features.email.value_counts()[:15].index
email

Index(['nun', 'hotmail', 'gmail', 'yahoo', 'live', 'aol', 'btinternet',
       'outlook', 'sky', 'icloud', 'googlemail', 'msn', 'ntlworld', 'talktalk',
       'virginmedia'],
      dtype='object')

In [242]:
def email_class(x):
  if x in email:
    return x
  else:
    return 'Others'

In [243]:
df_features['email']=df_features['email'].apply(lambda x: email_class(x))

In [244]:
df_features['email'][:3]

0    nun
1    nun
2    nun
Name: email, dtype: object

### 2.4 Payment Type

In [245]:
df_features.payment_type.value_counts()

paypalExpress    17327
credit_card       2288
Name: payment_type, dtype: int64

In [246]:
payment_type=df_features.payment_type.value_counts().index
payment_type

Index(['paypalExpress', 'credit_card'], dtype='object')

### 2.5 product_price

In [247]:
print(type(df_features.product_price[0]))
type(float(df_features.product_price[0]))

<class 'NoneType'>


TypeError: float() argument must be a string or a number, not 'NoneType'

In [248]:
string='2,500.00'
string.replace(',','')

'2500.00'

In [130]:
def product_price_replace(x):
  x=x.replace(',','')
  try:
    return(float(x))
  except ValueError:
    return 0

In [131]:
# eg.
string='2,500.00pp'
product_price_replace(string)

0

In [135]:
df_features['product_price']=df_features['product_price'].apply(lambda x: product_price_replace(x) if not pd.isnull(x) else 0)

### 2.6 product_save

In [118]:
print(df_features.product_save[0])
print(type(df_features.product_save[0]))

86%
<class 'str'>


In [124]:
def product_save_clean(x):
  x=x.replace('%','')
  try:
    return(float(x))
  except ValueError:
    return '0'

In [125]:
product_save_clean('50% de')

'0'

In [136]:
df_features['product_save']=df_features['product_save'].apply(lambda x: product_save_clean(x) if not pd.isnull(x) else 0)

### 2.7 total

In [143]:
print(df_features.total[0])
print(type(df_features.total[0]))

69.00
<class 'str'>


In [144]:
def total(x):
  try:
    return(float(x))
  except ValueError:
    return '0'

In [147]:
df_features['total']=df_features['total'].apply(lambda x: total(x) if not pd.isnull(x) else 0)

### 2.8 data_browser_family

In [164]:
data_browser_family=df_features.data_browser_family.value_counts()[:13].index
data_browser_family

Index(['Mobile Safari', 'Chrome', 'Samsung Internet', 'IE', 'Edge', 'Safari',
       'Firefox', 'Facebook', 'Amazon Silk', 'Chrome Mobile iOS', 'IE Mobile',
       'Firefox Mobile', 'Opera'],
      dtype='object')

In [166]:
def data_browser_family_class(x):
  if x in data_browser_family:
    return x
  else:
    return 'Others'

In [167]:
df_features['data_browser_family']=df_features['data_browser_family'].apply(lambda x: data_browser_family_class(x))

In [169]:
df_features['data_browser_family'][:3]

0    Mobile Safari
1    Mobile Safari
2    Mobile Safari
Name: data_browser_family, dtype: object

### 2.9 data_os_family

In [174]:
data_os_family=df_features.data_os_family.value_counts().index
data_os_family

Index(['iOS', 'Android', 'Windows', 'Mac OS X', 'Other', 'Linux', 'Ubuntu',
       'BlackBerry OS', 'Fedora', 'BlackBerry Tablet OS', 'Symbian OS'],
      dtype='object')

### 2.10 data_device_brand

In [178]:
data_device_brand=df_features.data_device_brand.value_counts()
data_device_brand

Apple      117754
Generic     53546
Samsung     24506
HTC          1625
LG           1422
Lenovo        891
Huawei        495
Archos         26
Sony           16
Spider         14
Name: data_device_brand, dtype: int64

In [180]:
%%time
with open( "../Data/Feature_Engineering/df_features_TimeCate.p", "wb" ) as f:
    pickle.dump(df_features, f)

CPU times: user 421 ms, sys: 121 ms, total: 542 ms
Wall time: 541 ms


### Redefine all category columns

In [11]:
%%time
with open( "../Data/Feature_Engineering/df_features_TimeCate.p", "rb" ) as f:
    df_features_TimeCate=pickle.load(f)

CPU times: user 244 ms, sys: 108 ms, total: 352 ms
Wall time: 352 ms


In [12]:
df_features_TimeCate[:3]

,num_sess,basketPaid,cart_firstTime,city,country,date_time,email,email_click_through,payment_type,product_price,product_save,product_time_left,product_viewed,total,dwell_time,weekday,days,month,time_of_day,cart_time,num_browsing,data_browser_family,data_browser_version_string,data_os_family,data_os_version_string,data_device_family,data_device_model,data_device_brand,referrer_binary
0,1,False,1.496607e+09,Others,GB,1.496603e+09,gmail,False,None,69.0,86,13999.0,1.496603e+09,69.0,3612.0,6,4,6,68800,11.0,4,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
1,2,False,1.496915e+09,Others,GB,1.496911e+09,gmail,False,None,49.0,51,224461.0,0.000000e+00,49.0,3709.0,3,8,6,31031,1.0,15,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0
2,3,False,1.495467e+09,Others,GB,1.495464e+09,gmail,False,None,99.0,51,289635.0,0.000000e+00,99.0,3625.0,0,22,5,52338,18.0,7,Mobile Safari,10.0,iOS,10.3.2,iPhone,iPhone,Apple,0


In [13]:
df_features_TimeCate.basketPaid[0]

'False'

In [15]:
df_features_TimeCate.basketPaid.value_counts()
label=df_features_TimeCate.basketPaid

False    279793
True          5
Name: basketPaid, dtype: int64

In [21]:
def label_class(x):
  if x == 'True':
    return 1
  else:
    return 0

In [22]:
df_features_TimeCate.basketPaid=df_features_TimeCate.basketPaid.apply(lambda x: label_class(x))

In [23]:
df_features_TimeCate.basketPaid[:3]

0    0
1    0
2    0
Name: basketPaid, dtype: int64

In [24]:
df_features_TimeCate=df_features_TimeCate.drop(['data_browser_version_string','data_os_version_string',
                           'data_device_family','data_device_model'],axis=1)

In [25]:
cols_to_transform = [ 'city','country','email','email_click_through','payment_type',
                     'data_browser_family','data_os_family','data_device_brand']
df_with_dummies = pd.get_dummies(df_features_TimeCate, columns = cols_to_transform)


In [26]:
print(df_with_dummies.shape)
df_with_dummies[:3]

(279798, 3245)


,num_sess,basketPaid,cart_firstTime,date_time,product_price,product_save,product_time_left,product_viewed,total,dwell_time,weekday,days,month,time_of_day,cart_time,num_browsing,referrer_binary,city_A Coruña,city_Abbots Langley,city_Aberaeron,city_Abercynon,city_Aberdalgie,city_Aberdare,city_Aberdeen,city_Abergavenny,city_Abergele,city_Abermule,city_Abernethy,city_Abertillery,city_Aberystwyth,city_Abingdon,city_Abingdon-on-thames,city_Aboyne,city_Abram,city_Abu Dhabi,city_Accra,city_Accrington,city_Acton,city_Addington,city_Addlestone,city_Adeje,city_Adelaide,city_Aguadulce,city_Ahmedabad,city_Airdrie,city_Al Ain,city_Albalat Dels Sorells,city_Albany,city_Albany Creek,city_Albrighton,city_Alcester,city_Alcorcón,city_Aldeburgh,city_Aldershot,city_Aldridge,city_Alexandria,city_Alford,city_Alfreton,city_Algoz,city_Algés,city_Alicante,city_Alkborough,city_Alloa,city_Almada,city_Alness,city_Alnwick,city_Alpharetta,city_Alresford,city_Alsager,city_Alston,city_Altamonte Springs,city_Alton,city_Altrincham,city_Alva,city_Alvechurch,city_Ambrogio,city_Amcotts,city_Amersham,city_Amesbury,city_Amlwch,city_Ammanford,city_Ampthill,city_Amstelveen,city_Amsterdam,city_Anapa,city_Andover,city_Angmering,city_Ann Arbor,city_Annan,city_Anniesland,city_Anstey,city_Anstruther,city_Antalya,city_Antibes,city_Antrim,city_Antwerp,city_Antwerpen,city_Apo,city_Appleby,city_Appley Bridge,city_Aramoho,city_Arbil,city_Arborfield,city_Arbroath,city_Ardleigh,city_Ardrossan,city_Arganda,city_Arlesey,city_Armadale,city_Armagh,city_Armthorpe,city_Arnold,city_Arona,city_Arrecife,city_Arucas,city_Arundel,city_Ascot,city_Ash,city_Ash Vale,city_Ashbourne,city_Ashburton,city_Ashby,city_Ashby De La Zouch,city_Ashby Magna,city_Ashfield,city_Ashford,city_Ashington,city_Ashperton,city_Ashtead,city_Ashton,city_Ashton-under-lyne,city_Ashurst,city_Askam,city_Astley,city_Aston,city_Aston Clinton,city_Astwood Bank,city_Athens,city_Atherstone,city_Atherton,city_Athína,city_Atlanta,city_Attard,city_Attleborough,city_Auchterarder,city_Auchtermuchty,city_Auckland,city_Aude,city_Aughnacloy,city_Aurora,city_Aveley,city_Avilés,city_Avonmouth,city_Awsworth,city_Axbridge,city_Axminster,city_Aycliffe,city_Aydin,city_Aydogan,city_Aylesbeare,city_Aylesbury,city_Aylesford,city_Aylsham,city_Ayot Saint Lawrence,city_Ayr,city_Bacup,city_Badajoz,city_Badalona,city_Bagillt,city_Bagshot,city_Baillieston,city_Bakewell,city_Baku,city_Balbeggie,city_Balcombe,city_Baldock,city_Balerno,city_Balham,city_Ballincollig,city_Ballingry,city_Balloch,city_Ballycarry,city_Ballyclare,city_Ballycullen,city_Ballymena,city_Ballymoney,city_Ballynahinch,city_Ballywalter,city_Balmullo,city_Bamber Bridge,city_Bamford,city_Bampton,city_Banbridge,city_Banbury,city_Banchory,city_Bandar Seri Begawan,city_Banff,city_Bangalore,city_Bangkok,city_Bangor,city_Bankfoot,city_Banstead,city_Bapchild,city_Barcelona,city_Barceloneta,city_Barcelos,city_Bardon Mill,city_Barford,city_Bargoed,city_Barking,city_Barkingside,city_Barkway,city_Barlborough,city_Barmouth,city_Barnard Castle,city_Barnes,city_Barnet,city_Barnetby,city_Barnham,city_Barningham,city_Barnoldswick,city_Barnsley,city_Barnstaple,city_Barrhead,city_Barrington,city_Barrow,city_Barrow In Furness,city_Barrow Upon Humber,city_Barrow Upon Soar,city_Barrowford,city_Barry,city_Bartley,city_Barton,city_Barton Under Needwood,city_Barton Upon Humber,city_Barwell,city_Basildon,city_Basingstoke,city_Batcombe,city_Bath,city_Bathgate,city_Batley,city_Battle,city_Baydon,city_Beaconsfield,city_Bearsden,city_Beaupréau,city_Beaverton,city_Bebington,city_Beccles,city_Beckenham,city_Beckermet,city_Beckton,city_Bedale,city_Bedford,city_Bedlington,city_Bedminster,city_Bedwas,city_Bedworth,city_Beenham,city_Beeston,city_Beith,city_Belfast,city_Belgooly,city_Bellevue,city_Bellshill,city_Belpasso,city_Belper,city_Belton,city_Belval,city_Belvedere,city_Bembridge,city_Benalmádena,city_Benenden,city_Benfleet,city_Benidorm,city_Benson,city_Bentley,city_Bere Regis,city_Berkeley,city_Berkeley Heigh

In [27]:
%%time
with open( "../Data/Feature_Engineering/df_with_dummies.p", "wb" ) as f:
    pickle.dump(df_with_dummies, f)

CPU times: user 399 ms, sys: 1.42 s, total: 1.82 s
Wall time: 1.82 s


---
# Label: Basket Paid

**Load df_final:**

In [47]:
%%time
with open( "../Data/data_final.p", "rb" ) as f:
    data_final=pickle.load(f)

CPU times: user 4.94 s, sys: 2.3 s, total: 7.24 s
Wall time: 7.24 s


In [48]:
data_final[['id','num_sess','basket_paid_browsing','basket_paid_cart']].loc[range(3566,3575)]

,id,num_sess,basket_paid_browsing,basket_paid_cart
3566,182,3,None,False
3567,182,3,None,False
3568,183,1,None,True
3569,183,1,None,True
3570,183,1,None,True
3571,183,1,None,True
3572,183,1,True,True
3573,183,2,None,True
3574,183,2,None,True


## 1. Investigate #True in Browse and Cart data

**Count #True from: **
<br>**1. basket_paid_browsing **
<br>**2. basket_paid_cart**

In [49]:
print('Following count is for df_data:')
print('count #True for basket_paid_browsing: ', sum(data_final['basket_paid_browsing']==True))
print('count #True for basket_paid_cart:     ', sum(data_final['basket_paid_cart']=='True'))

Following count is for df_data:
count #True for basket_paid_browsing:  20458
count #True for basket_paid_cart:      376486


**Get id_sess_true Table:**
<br>**This table would give you all (id, num_sess) with basket_paid_browsing = true.**

### For browse data:

In [50]:
# get 'id' and 'num_sess' column for basket_paid_browsing
id_sess_true = data_final[data_final['basket_paid_browsing']==True][['id','num_sess']]
print('#(id, num_sess) with label = True : ',len(id_sess_true))
print('\nid_sess table:')
id_sess_true[:2]

#(id, num_sess) with label = True :  20458

id_sess table:


,id,num_sess
2069,88,7
3572,183,1


In [51]:
print('drop duplicated term in id_sess_true and get shape: ', id_sess_true.drop_duplicates().shape)
print(len(id_sess_true),'!=',id_sess_true.drop_duplicates().shape[0])
print('Therefore, it is proved that there exists multi-buy within sessions')

drop duplicated term in id_sess_true and get shape:  (20037, 2)
20458 != 20037
Therefore, it is proved that there exists multi-buy within sessions


**True% analysis**

In [52]:
tmp=data_final.groupby(['id','num_sess'],sort=False).nth(0)
print('The shape of data_groupby with both T or F labels            :  ',tmp.shape)
print('Percentage of purchase by products (including multi-buy)) is :  ', 20458/tmp.shape[0]*100,'%')

The shape of data_groupby with both T or F labels            :   (220047, 28)
Percentage of purchase by products (including multi-buy)) is :   9.297104709448437 %


### For cart data:

In [53]:
# get 'id' and 'num_sess' column for basket_paid_browsing
id_sess_true = data_final[data_final['basket_paid_cart']=='True'][['id','num_sess']]
print('#(id, num_sess) with label = True (duplicated term dropped): ',len(id_sess_true.drop_duplicates()))
print('\nid_sess table:')
id_sess_true[:2]

#(id, num_sess) with label = True (duplicated term dropped):  33677

id_sess table:


,id,num_sess
2063,88,7
2064,88,7


**True% analysis**

In [54]:
tmp=data_final.groupby(['id','num_sess'],sort=False).nth(0)
print('The shape of data_groupby with both T or F labels : ',tmp.shape)
print('Percentage of purchase by sessions is             : ', 33677/tmp.shape[0]*100,'%')

The shape of data_groupby with both T or F labels :  (220047, 28)
Percentage of purchase by sessions is             :  15.30445768404023 %


## 2. Check inner join of label=true from cart & browse

In [55]:
# browse data:
id_sess_true_browse = data_final[data_final['basket_paid_browsing']==True][['id','num_sess']]
id_sess_true_browse_drop=id_sess_true_browse.drop_duplicates()
print('browse: shape of id_sess for sessions : ', id_sess_true_browse_drop.shape)
# cart data:
id_sess_true_cart = data_final[data_final['basket_paid_cart']=='True'][['id','num_sess']]
id_sess_true_cart_drop=id_sess_true_cart.drop_duplicates()
print('cart:   shape of id_sess for sessions : ', id_sess_true_cart_drop.shape)

browse: shape of id_sess for sessions :  (20037, 2)
cart:   shape of id_sess for sessions :  (33677, 2)


In [56]:
joint_id_sess=pd.merge(id_sess_true_browse_drop, id_sess_true_cart_drop, how='inner', on=['id','num_sess'])

In [57]:
print(joint_id_sess.shape)
print('Conclusion:')
print('Cart data indeed contains more information of basket_paid.', 
      '\nOnly 20037-20034 = 3 "true" in browse data are not included in cart data.')

(20034, 2)
Conclusion:
Cart data indeed contains more information of basket_paid. 
Only 20037-20034 = 3 "true" in browse data are not included in cart data.


In [136]:
print('In order to use product name etc. features still decide to use browse data to get information')

In order to use product name etc. features still decide to use browse data to get information


---

## 3. Locate multi-buy
duplicated rows in id_sess_true for browse data indicating multi-buy

In [67]:
id_sess_true = data_final[data_final['basket_paid_browsing']==True][['id','num_sess']]

In [68]:
id_sess_multiBuy=id_sess_true[id_sess_true.duplicated(keep=False)] # keep=False would return all duplicated rows 
print(id_sess_multiBuy.shape)
id_sess_multiBuy[:10]

(707, 2)


,id,num_sess
17073,602,2
17074,602,2
17129,602,4
17130,602,4
19409,646,2
19410,646,2
22513,745,4
22514,745,4
22515,745,4
22516,745,4


In [69]:
# id_sess_multiBuy0
id_sess_multiBuy0=id_sess_multiBuy.drop_duplicates()
id_sess_multiBuy0.reset_index(inplace=True)


In [70]:
multiBuy_table=pd.merge(id_sess_multiBuy0,data_final,how='inner',on=['id','num_sess'])
print(multiBuy_table.shape)
multiBuy_table.drop_duplicates().shape

(19503, 31)


(19503, 31)

In [71]:
multiBuy_table.columns

Index(['index', 'id', 'num_sess', 'session_id', 'finger_print', 'datetime',
       'days', 'referrer', 'page_url', 'product_viewed_browse', 'cart_started',
       'basket_paid_browsing', 'product_name', 'product_save', 'product_price',
       'product_time_left', 'email', 'Post_Code', 'cart_startTime',
       'product_viewed_cart', 'total', 'basket_paid_cart', 'payment_type',
       'email_click_through', 'cart_firstTime', 'cart_lastTime',
       'basketInfoTime', 'ip_address', 'user_agent', 'city', 'country'],
      dtype='object')

In [73]:
tmp=multiBuy_table[['id','num_sess','datetime','session_id','referrer', 'page_url','product_viewed_browse','product_name','basket_paid_browsing','product_save','product_price','product_time_left']]

In [89]:
print(multiBuy_table.loc[[12,13]].drop_duplicates().loc[12,'product_name'])
print(multiBuy_table.loc[[12,13]].drop_duplicates().loc[13,'product_name'])

Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitchen Towel Bundle
Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitchen Towel Bundle


In [92]:
multiBuy_table.loc[[12,13,14,15,16,17]].drop_duplicates()

,index,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
12,17129,602,4,1ottm4i3gc5cd1auv2omnsq131,d4816edea186244a6c03df2c9c314d2a,2017-06-07 18:03:48,7,http%3A%2F%2Fwww.google.co.uk%2F,https%3A%2F%2Fwww.wowcher.co.uk%2Fppc%2Fuk%2Fb...,NaT,NaT,None,Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitc...,66%,14.99,25:4:39:20,gfleat@yahoo.co.uk,None,1496859660231,None,14.99,True,paypalExpress,False,2017-06-07 19:21:00,2017-06-07 19:21:14,1496859724,85.255.232.200,Mozilla/5.0+(Linux;+Android+7.0;+SM-G930F+Buil...,None,GB
13,17129,602,4,1ottm4i3gc5cd1auv2omnsq131,d4816edea186244a6c03df2c9c314d2a,2017-06-07 18:03:48,7,http%3A%2F%2Fwww.google.co.uk%2F,https%3A%2F%2Fwww.wowcher.co.uk%2Fppc%2Fuk%2Fb...,NaT,NaT,None,Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitc...,66%,14.99,25:4:39:20,gfleat@yahoo.co.uk,None,1496859660231,None,14.99,True,paypalExpress,False,2017-06-07 19:21:00,2017-06-07 19:21:14,1496859724,86.11.221.205,Mozilla/5.0+(Linux;+Android+7.0;+SM-G930F+Buil...,Flitwick,GB
14,17129,602,4,1ottm4i3gc5cd1auv2omnsq131,d4816edea186244a6c03df2c9c314d2a,2017-06-07 18:05:53,7,http%3A%2F%2Fwww.google.co.uk%2F,https%3A%2F%2Fwww.wowcher.co.uk%2Fppc%2Fuk%2Fb...,NaT,NaT,None,Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitc...,66%,14.99,25:4:39:20,gfleat@yahoo.co.uk,None,1496859660231,None,14.99,True,paypalExpress,False,2017-06-07 19:21:00,2017-06-07 19:21:14,1496859724,86.11.221.205,Mozilla/5.0+(Linux;+Android+7.0;+SM-G930F+Buil...,Flitwick,GB
15,17129,602,4,1ottm4i3gc5cd1auv2omnsq131,d4816edea186244a6c03df2c9c314d2a,2017-06-07 18:05:53,7,http%3A%2F%2Fwww.google.co.uk%2F,https%3A%2F%2Fwww.wowcher.co.uk%2Fppc%2Fuk%2Fb...,NaT,NaT,None,Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitc...,66%,14.99,25:4:39:20,gfleat@yahoo.co.uk,None,1496859660231,None,14.99,True,paypalExpress,False,2017-06-07 19:21:00,2017-06-07 19:21:14,1496859724,85.255.232.200,Mozilla/5.0+(Linux;+Android+7.0;+SM-G930F+Buil...,None,GB
16,17129,602,4,1ottm4i3gc5cd1auv2omnsq131,d4816edea186244a6c03df2c9c314d2a,2017-06-07 18:07:06,7,http%3A%2F%2Fwww.google.co.uk%2F,https%3A%2F%2Fwww.wowcher.co.uk%2Fppc%2Fuk%2Fb...,NaT,NaT,None,Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitc...,66%,14.99,25:4:39:20,gfleat@yahoo.co.uk,None,1496859660231,None,14.99,True,paypalExpress,False,2017-06-07 19:21:00,2017-06-07 19:21:14,1496859724,85.255.232.200,Mozilla/5.0+(Linux;+Android+7.0;+SM-G930F+Buil...,None,GB
17,17129,602,4,1ottm4i3gc5cd1auv2omnsq131,d4816edea186244a6c03df2c9c314d2a,2017-06-07 18:07:06,7,http%3A%2F%2Fwww.google.co.uk%2F,https%3A%2F%2Fwww.wowcher.co.uk%2Fppc%2Fuk%2Fb...,NaT,NaT,None,Sooo Soft 54 Rolls Toilet Paper & 24 Roll Kitc...,66%,14.99,25:4:39:20,gfleat@yahoo.co.uk,None,1496859660231,None,14.99,True,paypalExpress,False,2017-06-07 19:21:00,2017-06-07 19:21:14,1496859724,86.11.221.205,Mozilla/5.0+(Linux;+Android+7.0;+SM-G930F+Buil...,Flitwick,GB


In [93]:
tmp[tmp['id']==602][tmp[tmp['id']==602]['num_sess']==4]

In [65]:
tmp[tmp['id']==602].shape

(62, 12)

In [94]:
tmp[tmp['id']==602]['num_sess']==2

# Appendix

**inner join id_sess_true and df_final**
<br>**to get df_browse_true from**

In [149]:
# inner join id_sess_true and df:
id_sess_true = data_final[data_final['basket_paid_browsing']==True][['id','num_sess']]
df_browse_true=pd.merge(id_sess_true,data_final,'inner', on=['id','num_sess'])

print('shape of df_browse_true original       :', df_browse_true.shape)
print('shape of df_browse_true drop duplicates:', df_browse_true.drop_duplicates().shape)

print('----------------------------------------------------------------------')

# original data:
print('#True for df_browse_true original       :', (df_browse_true[['basket_paid_browsing']]==True).sum()[0])

# drop duplicates and redo:
df_browse_true_drop=df_browse_true.drop_duplicates()[['basket_paid_browsing']]==True
print('#True for df_browse_true drop duplicates:', df_browse_true_drop.sum()[0])

shape of df_browse_true original       : (409098, 30)
shape of df_browse_true drop duplicates: (362369, 30)
----------------------------------------------------------------------
#True for df_browse_true original       : 21916
#True for df_browse_true drop duplicates: 20458


In [30]:
df_browse_true_drop=df_browse_true.drop_duplicates()
print('')
df_browse_true_drop[:3]

,id,num_sess,session_id,finger_print,datetime,days,referrer,page_url,product_viewed_browse,cart_started,basket_paid_browsing,product_name,product_save,product_price,product_time_left,email,Post_Code,cart_startTime,product_viewed_cart,total,basket_paid_cart,payment_type,email_click_through,cart_firstTime,cart_lastTime,basketInfoTime,ip_address,user_agent,city,country
0,88,7,18yrzevjvtdg12jfmdilomamv,27837983657a0de6b3a32ac9a95c22a3,2017-06-10 09:08:47,10,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,2017-06-10 09:08:47.019,NaT,None,Lace Up Invisible Push Up Bra - 4 Colours!,87%,7.99,01:13:51:12,None,joan1leech@yahoo.co.uk,1497085802688,None,7.99,True,None,False,2017-06-10 10:10:03,2017-06-10 10:12:54,1497086107,195.147.100.114,Mozilla/5.0+(Windows+NT+5.1;+rv:52.0)+Gecko/20...,None,GB
1,88,7,18yrzevjvtdg12jfmdilomamv,27837983657a0de6b3a32ac9a95c22a3,2017-06-10 09:10:03,10,,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,NaT,2017-06-10 09:10:02.688,None,Lace Up Invisible Push Up Bra - 4 Colours!,87%,7.99,01:13:51:12,None,joan1leech@yahoo.co.uk,1497085802688,None,7.99,True,None,False,2017-06-10 10:10:03,2017-06-10 10:12:54,1497086107,195.147.100.114,Mozilla/5.0+(Windows+NT+5.1;+rv:52.0)+Gecko/20...,None,GB
2,88,7,18yrzevjvtdg12jfmdilomamv,27837983657a0de6b3a32ac9a95c22a3,2017-06-10 09:12:12,10,https%3A%2F%2Fwww.wowcher.co.uk%2Fdeal%2Fshop%...,https%3A%2F%2Fsecure.wowcher.co.uk%2Fcheckout%...,NaT,2017-06-10 09:12:11.368,None,Lace Up Invisible Push Up Bra - 4 Colours!,87%,7.99,01:13:51:12,None,joan1leech@yahoo.co.uk,1497085802688,None,7.99,True,None,False,2017-06-10 10:10:03,2017-06-10 10:12:54,1497086107,195.147.100.114,Mozilla/5.0+(Windows+NT+5.1;+rv:52.0)+Gecko/20...,None,GB


In [26]:
# task 1: is true only in final row of session?
# task 2: is it possible to have multiple true for same session?

task 1: is true only in final row of session?



For basket_paid_browsing:

In [38]:
df_browse_sort = df_browse_drop.sort_values(['id','num_sess','basket_paid_browsing']
                                        ,ascending=['True','True','False'])
df_browse_groupby=df_browse_sort.groupby(['id','num_sess'],sort=False).nth(0)

print(df_browse_groupby.shape)
count_true=sum(df_browse_groupby.basket_paid_browsing.apply(lambda x: x==True))
print('#True in df_browse_groupby (with ascending = [T,T,F]):')
print(count_true)



(20037, 28)
#True in df_browse_groupby (with ascending = [T,T,F]):
20037


In [39]:
sum(df_browse_groupby.basket_paid_browsing.apply(lambda x: x==True))

20037

In [29]:
Df_browse_sort=df_browse_drop.sort_values(['id','num_sess','basket_paid_browsing']
                                                      ,ascending=['True','True','True'])
Df_browse_groupby=Df_browse_sort.groupby(['id','num_sess'],sort=False).nth(0)


count_true = sum(Df_browse_groupby.basket_paid_browsing.apply(lambda x: x==True))
print('#True in df_browse_groupby (with ascending = [T,T,T]):')
print(count_true)


#True in df_browse_groupby (with ascending = [T,T,T]):
20037


---
**Test on id_sess_full, which has all columns of df_browse**

In [32]:
df_browse_sort=df_browse_drop.sort_values(['id','num_sess','basket_paid_browsing']
                                                      ,ascending=['True','True','False'])

# id_sess_full is all rows having true label (also having all columns)
id_sesee_full=df_browse_sort[df_browse_sort['basket_paid_browsing']==True]

print('shape of id_sess_full: ',id_sesee_full.shape)
print('shape of id_sess_full drop_duplicates： ',id_sesee_full.drop_duplicates().shape)

shape of id_sess_full:  (20458, 30)
shape of id_sess_full drop_duplicates：  (20458, 30)


In [41]:
df_browse_merge_full = pd.merge(id_sesee_full,data_sort_basket_paid_browsing
                                             ,how='inner', on=['id','num_sess'] ,suffixes=('','_y'))
print('df_browse_merge_full.shaped:\n',df_browse_merge_full.shape)
print('df_browse_merge_full.drop_duplicates().shape:\n',df_browse_merge_full.drop_duplicates().shape)
print('So there is no duplicated terms.')

df_browse_merge_full.shaped:
 (409098, 58)
df_browse_merge_full.drop_duplicates().shape:
 (409098, 58)
So there is no duplicated terms.


In [42]:
df_browse_groupby_full=df_browse_merge_full.groupby(['id','num_sess'],sort=False).nth(0)

print('#True of groupby:',sum(df_browse_groupby_full.basket_paid_browsing.apply(lambda x: x==True)))

#True of groupby: 20037


In [52]:
print('!=20458\nso it seems groupby would also decrease true label,implying there would be multi-purchase during session.')

!=20458
so it seems groupby would also decrease true label,implying there would be multi-purchase during session.


---

**Try to find multi-purchase**

In [69]:
df_browse_sort = df_browse_drop.sort_values(['id','num_sess','basket_paid_browsing']
                                        ,ascending=['True','True','False'])


In [117]:
tmp=df_browse_sort[['id','num_sess','basket_paid_browsing']].groupby(['id','num_sess'],sort=False).count()
print(tmp.sum())
tmp[:4]

basket_paid_browsing    20458
dtype: int64


basket_paid_browsing
id  num_sess                      
88  7                            1
183 1                            1
    2                            1
    3                            1

In [128]:
# tmp!=1
multi_buy=tmp[tmp['basket_paid_browsing']!=1]
multi_buy

basket_paid_browsing
id    num_sess                      
602   2                            2
      4                            2
646   2                            2
745   4                            4
923   1                            2
1027  2                            4
1055  2                            2
1182  4                            2
1440  1                            4
2155  5                            2
2164  3                            2
2271  5                            2
2871  1                            2
2875  1                            2
2963  1                            2
4145  2                            2
4248  2                            2
4409  1                            2
5066  7                            3
5493  4                            2
5529  1                            3
      2                            2
5581  1                            2
5677  2                            2
5867  3                            2
6032  4                            2
6542  2                            4
      4                            4
      9                            3
6573  1                            2
...                              ...
47120 2                            2
47194 1                            2
      3                            2
47221 2                            2
47330 4                            2
47368 4                            2
47410 2                            2
47420 3                            2
47577 1                            3
47675 2                            2
47732 1                            2
47734 1                            2
48198 3                            4
48514 1                            2
48800 2                            6
48981 1                            2
49319 1                            2
49856 11                           2
      12                           4
      23                           4
50113 3                            2
50159 2                            2
50777 1                            2
51396 4                            2
51474 5                            3
51568 3                            2
51606 1                            2
51824 3                            2
51902 1                            2
51979 1                            2

[286 rows x 1 columns]

In [ ]:
df_browse_sort = df_browse_drop.sort_values(['id','num_sess','basket_paid_browsing']
                                        ,ascending=['True','True','False'])
df_browse_groupby=df_browse_sort.groupby(['id','num_sess'],sort=False).count('basket_paid_browsing')

print('#True in df_browse_groupby (with ascending = [T,T,F]):')
print(sum(df_browse_groupby.basket_paid_browsing.apply(lambda x: x==True)))

In [ ]:
cols_to_transform = [ 'a', 'list', 'of', 'categorical', 'column', 'names' ]
df_with_dummies = pd.get_dummies( columns = cols_to_transform )
#http://fastml.com/converting-categorical-data-into-numbers-with-pandas-and-scikit-learn/

In [ ]:
#You can convert them into integer codes by using the categorical datatype.
column = column.astype('category')
column_encoded = column.cat.codes
# https://stackoverflow.com/questions/34915813/convert-text-columns-into-numbers-in-sklearn